In [2]:
import traci
import random
import sumolib
import os
import collections
import networkx as nx
import numpy as np
from itertools import permutations
import traci
import time
from pyscipopt import Model
import pickle
import itertools
from IPython.display import clear_output

# 基于SUMO地图文件（.net.xml）生成图，并随机初始化起点和多个终点，生成剪枝后的子图

In [2]:
def convert_net_to_digraph_simple(net):
    # 创建一个有向图
    G = nx.DiGraph() 
    # 遍历所有边
    for edge in net.getEdges():
        # 遍历边的所有车道
        for outgoing_edge in edge.getOutgoing():
            segment_length=edge.getLength()
            segment_speed = edge.getSpeed()
            from_edge_id=edge.getID()
            to_edge_id=outgoing_edge.getID()
            G.add_edge(from_edge_id, to_edge_id, roadlength=segment_length, roadspeed=segment_speed,roadtime=segment_length/segment_speed)
    return G
def create_new_nodes(net,num_nodes):
    # 获取所有边
    edge_all = net.getEdges()
    create_list=[]
    # 随机选择一条边
    for i in range(num_nodes):
        edge = random.choice(edge_all)
        lane =random.choice(edge.getLanes())
        edge_length = edge.getLength()
        distance=random.uniform(2, max(5,edge_length-10))
        laneID=lane.getID()
        shape = edge.getShape()# 获取边起止点的坐标
        # 计算在当前线段上的位置
        ratio = distance / edge_length
        x = shape[0][0] + ratio * (shape[1][0] - shape[0][0])
        y = shape[0][1] + ratio * (shape[1][1] - shape[0][1])
        # 生成唯一的节点ID
        node_id =( f"Target_{i}")
        create_list.append([node_id, (x, y), laneID, distance])
    return create_list

def convert_net_to_digraph_simple(net):
    # 创建一个有向图
    G = nx.DiGraph() 
    # 遍历所有边
    for edge in net.getEdges():
        # 遍历边的所有车道
        for outgoing_edge in edge.getOutgoing():
            segment_length=edge.getLength()
            segment_speed = edge.getSpeed()
            from_edge_id=edge.getID()
            to_edge_id=outgoing_edge.getID()
            G.add_edge(from_edge_id, to_edge_id, roadlength=segment_length, roadspeed=segment_speed,roadtime=segment_length/segment_speed)
    return G

def convert_net_to_digraph(net,add_nodes):
    # 创建一个有向图
    G = nx.DiGraph() 
    # 遍历所有边
    for edge in net.getEdges():
        # 遍历边的所有车道
        for lane_index, lane in enumerate(edge.getLanes()):
            # 获取车道ID
            lane_node_id = lane.getID()
            # 遍历出边
            for ToEdge in edge.getOutgoing():
                # 获取连接列表
                conn_list=edge.getConnections(ToEdge)
                # 遍历连接
                for ToConnection in conn_list:
                    #获取车道ID
                    to_lane_node_id = ToConnection.getToLane().getID()
                     # 获取交通信号灯和连接索引
#                     print(dir(ToConnection))
                    tl = ToConnection.getTLSID()  # 获取交通信号灯ID
                    linkIndex = ToConnection.getTLLinkIndex()  # 获取连接索引
#                     print(f"tl ID:{tl}, index ID: {linkIndex}")
                    # 合并交通信号灯和连接索引
                    segment_tl = f"{tl}_{linkIndex}" if tl else None
                    # 定义边属性
                    segment_speed = edge.getSpeed()
                    segment_length= edge.getLength()
#                     print(to_lane_node_id)
                    # 在有向图中添加边
                    G.add_edge(lane_node_id, to_lane_node_id, roadlength=segment_length, roadspeed=segment_speed,roadtime=segment_length/segment_speed,trafficlight=segment_tl)
#     print(G.nodes())
    target_list=[]
    for i in range(len(add_nodes)):
        origin_node=add_nodes[i][2]
        distance=add_nodes[i][3]
        new_node_ID1=f"A_{origin_node}"
        new_node_ID2=f"B_{origin_node}"
        # 继承节点u的所有入边
#         print(origin_node)
        for predecessor in list(G.predecessors(origin_node)):
#             print(predecessor)
#             print(origin_node)
            segment_length=G[predecessor][origin_node]['roadlength']
            segment_speed=G[predecessor][origin_node]['roadspeed']
            G.add_edge(predecessor, new_node_ID1, roadlength=segment_length, roadspeed=segment_speed,roadtime=segment_length/segment_speed)
            G.remove_edge(predecessor, origin_node)

        # 继承节点u的所有出边
        for successor in list(G.successors(origin_node)):
            segment_length=G[origin_node][successor]['roadlength']
            segment_speed=G[origin_node][successor]['roadspeed']
            G.add_edge(new_node_ID2, successor, roadlength=segment_length, roadspeed=segment_speed,roadtime=segment_length/segment_speed)
            G.remove_edge(origin_node, successor)
        # 新增边(u1, u2)
        G.add_edge(new_node_ID1, new_node_ID2, roadlength=distance, roadspeed=segment_speed,roadtime=segment_length/segment_speed,trafficlight=None)
        target_list.append(new_node_ID2)
#     原节点的所有入边和出边都删除后，该节点自动删除
#     for i in range(len(add_nodes)):
#         origin_node=add_nodes[i][2]
#         # 删除原节点u
#         G.remove_node(origin_node)

    return G,target_list
def calculate_filted_path(G, source_node, target_node, length_cutoff_gain, node_cutoff_add,time_cutoff_gain):
    minPath_L = nx.dijkstra_path(G, source_node, target_node, weight='roadlength')
    L_minPath=nx.dijkstra_path_length(G, source_node, target_node, weight='roadlength')
    num_node_L=len(minPath_L)
    minPath_T = nx.dijkstra_path(G, source_node, target_node, weight='roadtime')
    T_minPath=nx.dijkstra_path_length(G, source_node, target_node, weight='roadtime')
    num_node_T=len(minPath_L)
    node_cutoff=node_cutoff_add+min(num_node_L,num_node_T)
    length_cutoff=length_cutoff_gain*L_minPath
    time_cutoff=time_cutoff_gain*T_minPath
    # 存储所有路径及其长度的列表
    path_lengths = []
    if length_cutoff_gain==1 and node_cutoff_add==0 and time_cutoff_gain==1:
        path_all=[minPath_L,minPath_T]
    else:    
        path_all=nx.all_simple_paths(G, source_node, target_node, cutoff=node_cutoff)
    # 生成所有简单路径
    for i, path in enumerate(path_all):
        # 计算路径长度
        path_length = sum(G[path[i]][path[i+1]].get('roadlength', None) for i in range(len(path) - 1))
        path_time = sum(G[path[i]][path[i+1]].get('roadtime', None)  for i in range(len(path) - 1))
        if length_cutoff> path_length and time_cutoff> path_time:
            # 将路径及其长度添加到列表中
            path_lengths.append(path)
        else:
            continue
    return path_lengths,minPath_L,minPath_T
def create_graph_from_paths(paths):
    G = nx.DiGraph()
    for path in paths:
        for i in range(len(path) - 1):
            # 获取路径中的每对节点
            u, v = path[i], path[i + 1]
            # 为图中的每对节点添加一条边，并设置权重
            if not G.has_edge(u, v):
                G.add_edge(u, v)
    return G
def create_graph_list(G,start_node,target_nodes,node_cutoff_add,length_cutoff_gain,time_cutoff_gain,indexed_targets=None):
#     target_nodes = list(range(start_node+1, num_nodes))
#     target_nodes = [str(num) for num in range(43,G.number_of_nodes())]
    if indexed_targets!=None:
        all_target_permutations=[indexed_targets]
#         print(1)
    else:      
        all_target_permutations = list(permutations(target_nodes))
    graph_list = []
    graph_union=nx.DiGraph()
    possible_path_L=[]
    possible_path_T=[]
    for permutation_i in all_target_permutations:
        source_node=start_node
        graph_row=[]
        possible_path_row_L=[]
        possible_path_row_T=[]
        for i in range(len(permutation_i)):
            target_node = permutation_i[i]
#             print(f'source node: {source_node}, target_node: {target_node}')
            paths,minPath_L,minPath_T = calculate_filted_path(G, source_node, target_node, length_cutoff_gain, node_cutoff_add,time_cutoff_gain)
            G_created=create_graph_from_paths(paths)
            source_node=permutation_i[i]
            graph_row.append(G_created)
            graph_union=nx.compose(graph_union, G_created)
#             print(f'Combined graph {graph_union} and created graph {G_created}')
            if i==0:
                possible_path_row_L=possible_path_row_L+minPath_L
                possible_path_row_T=possible_path_row_T+minPath_T
            else:
                possible_path_row_L=possible_path_row_L+minPath_L[1:]
                possible_path_row_T=possible_path_row_T+minPath_T[1:]
        if len(all_target_permutations)==1:
#             print('Indexed targets with 1 permutation')
            graph_list=graph_row
            possible_path_L=possible_path_row_L
            possible_path_T=possible_path_row_T
        else:
            graph_list.append(graph_row)
            possible_path_L.append(possible_path_row_L)
            possible_path_T.append(possible_path_row_T)
    return graph_list,graph_union,all_target_permutations,possible_path_L,possible_path_T



In [ ]:
net = sumolib.net.readNet("D:\\SUMO\\work\\rebuild2\\sourceCICT.net.xml")
G_simple=convert_net_to_digraph_simple(net)
generate_num=10
target_num=8
rd_nodes=create_new_nodes(net,generate_num+1)
G_original,generate_list=convert_net_to_digraph(net,rd_nodes)
# print(G_original.nodes())
# print(generate_list)
DJL_database=[]
DJT_database=[]
start_database=[]
target_database=[]
set_num=20*len(generate_list)
max_num=len(list(permutations(generate_list)))
print(f"set number of generation: {set_num}, max number of generation: {max_num}")
start_time = time.time()
start_node_list=[]
database_dic={}
node_cutoff_add=0
length_cutoff_gain=1
time_cutoff_gain=1
i=0
sample_counter=0
error_counter=0
free_counter=0
generate_set_simple=set()
for additional_node in generate_list:
    additional_node_simple=additional_node.split("_")[1]
    generate_set_simple.add(additional_node_simple)
generate_list_simple=list(generate_set_simple)
# generate_list_simple=['27-26', '19-26', '28-21', '36-37', '21-20', '9-17', '17-18', '33-40', '23-24', '30-37', '10-9']
print(generate_list_simple)
while i<min(set_num,max_num):
# for i in range(min(set_num,max_num)):
    data_single={}
    start_node_simple= random.choice(generate_list_simple)
    start_initial_counter=1
    while start_node_simple in start_node_list and start_initial_counter<10:
        start_node_simple= random.choice(generate_list_simple)
        start_initial_counter=start_initial_counter+1
    original_list = [item for item in generate_list_simple if item != start_node_simple]
    target_list = random.sample(original_list,target_num)
#     print(target_list)
    indexed_targets_simple=list(random.choice(list(permutations(target_list))))
    G_list,G_combined,_,possible_path_L,possible_path_T=create_graph_list(G_simple,start_node_simple,target_list,6,1.2,1.2,indexed_targets_simple)
    break_flag=False
    sample_counter=sample_counter+1
    if sample_counter%10==1:
        print(f"{sample_counter}th sample")
    for index_i in range(len(G_list)):
        G=G_list[index_i]
        if len(G.nodes())==0:
            print("空路径无法组成搜索图！")
            break_flag=True
            break
        try:
            cycle = nx.find_cycle(G)
#             print(f"Cycle found: {cycle} of vehicle {str(vehicle_id_index)} toward {control_dic[str(vehicle_id_index)]['targets'][index]} within {control_dic[str(vehicle_id_index)]['targets']}")
            break_flag=True
            error_counter=error_counter+1
            print("有环图不适用！ ")
            break
        except nx.NetworkXNoCycle:
            free_counter=free_counter+1
#             print("无环图！ ")
    if break_flag:
        continue
    data_single['Graph list']=G_list
    data_single['Combined graph']=G_combined
    data_single['Start']=start_node_simple
    data_single['Targets']=target_list
    data_single['Indexed Targets']=indexed_targets_simple
    data_single['Possible DJL']=possible_path_L
    data_single['Possible DJT']=possible_path_T
    database_dic[i]=data_single
    start_node_list.append(start_node_simple)
    i=i+1
end_time = time.time()      
elapsed_time = end_time - start_time
print(f"程序运行耗时: {elapsed_time:.6f} 秒, 共计采样 {sample_counter} 次，获得样本{i}例")

# 在生成后的数据库字典database_dic中随机初始化N辆车，各自路由选择其中的“Possible DJL”和“Possible DJT”，分别写入SUMO的路由文件（.rou.xml），用作baseline实验配置

In [3]:
def NX2SUMO_route_converted(node_path):
    if any(isinstance(i, list) for i in node_path):
        num_route=len(node_path)
    else:
        num_route=1
    edge_matrix=[]
#     print(num_route)
    for i in range(num_route):
        route=[]
        edge_id_last=[]
        path=node_path[i]
        for lane_id in path:
            # 检查是否为新增节点
            if lane_id.startswith('A_') or lane_id.startswith('B_'):
                # 提取最近边ID
                edge_id = lane_id.split('_')[1]
                if edge_id==edge_id_last:
                    continue
            else:
                # 提取边ID
                edge_id = lane_id.rsplit('_', 1)[0]
            route.append(edge_id)
            edge_id_last=edge_id
        edge_matrix.append(route)        
    return edge_matrix
def write_routes_to_xml(net, edge_matrix, output_path, num_vehicles=None, num_vehicle_types=3):
    """
    将生成的edge_matrix写入.rou.xml文件中，并生成N辆车辆，每辆车的类型随机化。
    
    参数:
    net (sumolib.net.Net): 通过sumolib读取的路网对象
    edge_matrix (list of lists): 二维列表，每个元素是一个edge_list
    output_path (str): 输出文件的路径
    num_vehicles (int): 生成的车辆数量，如果为None，则随机生成
    num_vehicle_types (int): 预定义的车辆类型数量
    """
    # 如果num_vehicles为None，则随机生成一个车辆数量
#     print(edge_matrix[:][0])
    if num_vehicles is None:
        num_vehicles = random.randint(1, 10)  # 生成1到10之间的随机整数
    
    # 生成预定义的车辆类型
    vehicle_types = []
    # 定义可能的guiShape值
    gui_shapes = ["passenger", "truck", "evehicle"]
    # 定义可能的laneChangeModel值
    lane_change_models = ["LC2013", "SL2015"]
    # 定义可能的carFollowModel值
    car_follow_models = ["PWagner2009", "IDM", "Krauss", "SmartSK", "W99", "CACC"]
    
    for i in range(num_vehicle_types):
        gain=random.uniform(0.5, 1.5)
        select_guishape=random.choice(gui_shapes)
        select_lanechange=random.choice(lane_change_models)
        select_carfollow=random.choice(car_follow_models)
#         vehicle_types.append(f"""
#     <vType id="type{i+1}" accel="{gain}" decel="{gain * 2}" sigma="{random.uniform(0.1, 0.9)}" tau="{2* gain**0.5}" length="{3 / gain}" maxSpeed="{30 + gain * 20}" guiShape="{select_guishape}" laneChangeModel="{select_lanechange}" carFollowModel="{select_carfollow}"/>
# """)
        vehicle_types.append(f"""
    <vType id="type{i+1}" accel="{gain}" decel="{gain * 2}" sigma="{random.uniform(0.1, 0.9)}" tau="{2* gain**0.5}" length="{3 / gain}" maxSpeed="{30 + gain * 20}" guiShape="{select_guishape}" laneChangeModel="LC2013" />
""")

        
    
    # 生成路线文件内容
    route_xml = """
<routes>
""" + ''.join(vehicle_types) + """
"""
    
    # 检查每个edge_list中的边是否存在，并且连接是否有效
    for edge_list in edge_matrix:
        for i in range(len(edge_list) - 1):
            current_edge = edge_list[i]
            next_edge = edge_list[i + 1]
            
            # 检查当前边是否存在
            if not net.hasEdge(current_edge):
                raise ValueError(f"边 {current_edge} 不在路网中")
            
            # 获取当前边和下一条边的对象
            current_edge_obj = net.getEdge(current_edge)
            next_edge_obj = net.getEdge(next_edge)
            # 检查连接是否存在
            outgoing_edges = current_edge_obj.getOutgoing()
            if next_edge_obj not in outgoing_edges:
                raise ValueError(f"边 {current_edge} 和边 {next_edge} 之间没有有效的连接")
#     print(edge_matrix[:][0])
    selected_route_list=[]
    selected_vehicle_list=[]
    for i in range(num_vehicles):
        selected_route_index=random.choice(range(len(edge_matrix)))
        selected_route_list.append(selected_route_index)
        selected_vehicle_index=random.choice(range(len(vehicle_types)))
        selected_vehicle_list.append(selected_vehicle_index)
        edge_list = edge_matrix[selected_route_index]  # 随机选择一个edge_list
        vehicle_type = vehicle_types[selected_vehicle_index]  # 随机选择一个车辆类型
        route_xml += f"""
    <vehicle id="{i}" type="{vehicle_type.split()[1].split('=')[1].strip('"')}" depart="0" color="1,0,0">
      <route edges="{' '.join(edge_list)}"/>
    </vehicle>
"""
    route_xml += """
</routes>
"""
    # 将内容写入文件
    with open(output_path, "w") as file:
        file.write(route_xml)
    print(f"路线文件已生成: {output_path}")
    return vehicle_types,edge_matrix,selected_route_list,selected_vehicle_list

def write_routes_to_xml_recurrent(output_path,vehicle_types,edge_matrix,selected_route_index,selected_vehicle_index):
    route_xml = """
<routes>
""" + ''.join(vehicle_types) + """
"""
    for j in range(len(selected_route_index)):
        i=selected_route_index[j]
        k=selected_vehicle_index[j]
        edge_list = edge_matrix[i]  # 随机选择一个edge_list
        vehicle_type = vehicle_types[k]  # 随机选择一个车辆类型
        route_xml += f"""
    <vehicle id="{j}" type="{vehicle_type.split()[1].split('=')[1].strip('"')}" depart="0" color="1,0,0">
      <route edges="{' '.join(edge_list)}"/>
    </vehicle>
"""
    route_xml += """
</routes>
"""
    # 将内容写入文件
    with open(output_path, "w") as file:
        file.write(route_xml)
    print(f"路线文件已生成: {output_path}")
        


In [ ]:
net = sumolib.net.readNet("D:\\SUMO\\work\\rebuild2\\sourceCICT.net.xml")     
vehicle_num=300
vehicle_type_num=10
test_paths_DJL=[]
test_paths_DJT=[]
test_paths_mix=[]
for i in range(len(database_dic)):
    test_paths_DJL.append(database_dic[i]['Possible DJL'])
    test_paths_DJT.append(database_dic[i]['Possible DJT'])
    test_paths_mix.append( random.choice( [ database_dic[i]['Possible DJL'],database_dic[i]['Possible DJT'] ]) )

output_path_DJL = "D:\\SUMO\\work\\rebuild2\\route_DJL.rou.xml"       
edge_matrix_DJL=NX2SUMO_route_converted(test_paths_DJL)
vehicle_types,_,selected_route_list,selected_vehicle_list=write_routes_to_xml(net, edge_matrix_DJL, output_path_DJL, vehicle_num, vehicle_type_num)
output_path_DJT = "D:\\SUMO\\work\\rebuild2\\route_DJT.rou.xml"       
edge_matrix_DJT=NX2SUMO_route_converted(test_paths_DJT)
write_routes_to_xml_recurrent(output_path_DJT, vehicle_types, edge_matrix_DJT, selected_route_list,selected_vehicle_list)
output_path_mix = "D:\\SUMO\\work\\rebuild2\\route_mix.rou.xml"       
edge_matrix_mix=NX2SUMO_route_converted(test_paths_mix)
write_routes_to_xml_recurrent(output_path_mix, vehicle_types, edge_matrix_mix, selected_route_list,selected_vehicle_list)
database_baseline={'Possible DJLs list':edge_matrix_DJL,'Possible DJLs list':edge_matrix_DJL,'Possible mixed paths':edge_matrix_mix,'Possible vehicle types list':vehicle_types,'Routes index list': selected_route_list,'Vehicles index list': selected_vehicle_list}


In [181]:

output_file_path = r"D:\\SUMO\\work\\rebuild2\\basic_database.pkl"  # 替换为你想要保存的文件名
with open(output_file_path, 'wb') as file:
    pickle.dump((database_dic,database_baseline), file)

print(f"数据已保存到：{output_file_path}")

数据已保存到：D:\\SUMO\\work\\rebuild2\\basic_database.pkl


In [4]:

output_file_path = r"D:\\SUMO\\work\\rebuild2\\basic_database.pkl"  # 替换为你想要保存的文件名
# 使用pickle从文件加载列表
with open(output_file_path, 'rb') as file:
    database_dic,database_baseline = pickle.load(file)
    
print(f"{output_file_path}数据已加载")

D:\\SUMO\\work\\rebuild2\\basic_database.pkl数据已加载


# 基于SCIP的排队长度最小车道规划

In [5]:
def lane_change_cost(current_lane,target_lane,obstacle_position,obstacle_target):
    basic_change_cost=np.abs(current_lane-target_lane)
    cross_collision=0
    conflict_collision=0
    for i in range(len(obstacle_position)):
        if obstacle_position[i] == target_lane and obstacle_target[i] == current_lane:
            cross_collision += 1
        if obstacle_position[i] == target_lane and obstacle_target[i] == target_lane:
            conflict_collision += 1
    collision_cost=cross_collision+conflict_collision
    total_cost=basic_change_cost+collision_cost
#     print(f"basic cost: {basic_change_cost}, collision cost: {collision_cost}, total cost: {total_cost}")
    return total_cost
def lane_assignment_optimization(assign_position,obstacle_position,obstacle_target,available_target,total_lane):
    cost_matrix=[]
    decision_matrix=[]
    model = Model("Lane Assignment")
    for i in range(len(available_target)):
        cost_row=[]
        decision_row=[]
        current_lane=assign_position[i]
#         print(available_target[i])
        for j in available_target[i]:
#             print(f"current lane in available horizon: {j}")
            cost_current=lane_change_cost(current_lane,j,obstacle_position,obstacle_target)
            cost_row.append(cost_current)
            decision_row.append( model.addVar(vtype="B", name=f"x({i},{j})"))
        decision_matrix.append(decision_row)
        cost_matrix.append(cost_row)
#     print(f"decision dimension: {len(decision_matrix)}")
#     print(f"cost matrix: {cost_matrix}")
    z= model.addVar(vtype="M", name=f"platoon_length")
    for i in range(len(available_target)):  # 每个车辆恰好被分配到一个车道
        model.addCons(sum(decision_matrix[i][j] for j in range(len(available_target[i]))) == 1)  
    for i in total_lane: # 隐变量大于等于每个车道预期的编队总长度
        constant_platoon=obstacle_target.count(i)
        model.addCons(z>=constant_platoon+sum(decision_matrix[j][available_target[j].index(i)] for j in range(len(available_target)) if i in available_target[j])) 
    model.setObjective(2*z+0.5*sum(cost_matrix[i][j] * decision_matrix[i][j] for i in range(len(available_target)) for j in range(len(available_target[i]))), "minimize" )  # 设置目标函数：最小化队列长度+换道成本 
    model.optimize()  # 求解优化模型
    assign_target=[]
    for i in range(len(available_target)):
        for j in range(len(available_target[i])):
            if model.getVal(decision_matrix[i][j]) > 0.5:
                assign_target.append(available_target[i][j])
    return assign_target

# 离线字典生成（不需要SUMO和TraCI）

In [6]:
def two_step_decision_list_generation(G):
    action_dict={}
    action_detail_dict={}
    for current_node in G.nodes():
        action_list=[]
        current_action_list=[]
        if len(list(G.successors(current_node)))==0:
            action_list.append([None,None])
            action_dict[current_node]=action_list
            current_action_list.append(None)
            action_detail_dict[current_node]=current_action_list
            continue
        for next_node in G.successors(current_node):
            current_action_list.append(next_node)
            if len(list(G.successors(next_node)))==0:
                action_list.append([next_node,None])
                break
            for next_next_node in G.successors(next_node):
                action_list.append([next_node,next_next_node])
        action_detail_dict[current_node]=current_action_list
        action_dict[current_node]=action_list
    return action_dict,action_detail_dict

def control_dictionary_initialize(G_total_dic,targets_dic,vehicle_ids):
    control_dict={}
    for vehicle_i in vehicle_ids:
        current_dict={}
        indexed_target=targets_dic[vehicle_i]
        action_list_single_S2T=[]
        action_detail_single_S2T=[]
        for index in range(len(indexed_target)):
            G=G_total_dic[vehicle_i][index]
            action_dict,action_detail_dict=two_step_decision_list_generation(G)
            action_list_single_S2T.append(action_dict)
            action_detail_single_S2T.append(action_detail_dict)
        current_dict['uk']=None
        current_dict['Tlane']=None
        current_dict['detailed action dictionary']=action_detail_single_S2T
        current_dict['action list']=action_list_single_S2T
        current_dict['targets']=indexed_target
        current_dict['Route']=[]
        current_dict['recorded route']=[]
        current_dict['stage']=0
        control_dict[vehicle_i]=current_dict
    return control_dict

def lane_set_generation(net):
    all_edges=net.getEdges()
    search_dic={} # 从edge1出发至edge2，可选择出发的edge1车道序号
    possible_dic={} # 从edge1出发至edge2，从edge1的车道i出发，可能到达的edge2车道序号
    for edge in all_edges:
        source_edge_id=edge.getID()
        all_outgoing=edge.getOutgoing()
        search_dic[source_edge_id]={}
        possible_dic[source_edge_id]={}
        for ic_edge in all_outgoing:
            target_edge_id=ic_edge.getID()
            connections=all_outgoing[ic_edge]
            constrains_list=set()
            possible_dic[source_edge_id][target_edge_id]={}
            for connection_i in connections:
                from_lane=connection_i.getFromLane().getID()
                to_lane=connection_i.getToLane().getID()
                fromlane_index=int(from_lane.split('_')[-1])
                tolane_index=int(to_lane.split('_')[-1])
                constrains_list.add(fromlane_index)
                possible_dic[source_edge_id][target_edge_id][fromlane_index]=set()
            for connection_i in connections:
                from_lane=connection_i.getFromLane().getID()
                to_lane=connection_i.getToLane().getID()
                fromlane_index=int(from_lane.split('_')[-1])
                tolane_index=int(to_lane.split('_')[-1])
                possible_dic[source_edge_id][target_edge_id][fromlane_index].add(tolane_index)
            search_dic[source_edge_id][target_edge_id]=list(constrains_list)   
    edge_search_dic={'Possible from lanes':search_dic,'Possible to lanes':possible_dic}
    return edge_search_dic


seleted_indexes_list=database_baseline['Routes index list']
# print(seleted_indexes_list)
G_total_dict={}
targets_dict={}
vehicle_ids=[]
for iteration_index in range(len(seleted_indexes_list)):
    vehicle_index=seleted_indexes_list[iteration_index]
    vehicle_id=str(iteration_index)
    current_data_index=seleted_indexes_list[vehicle_index]
    G_list=database_dic[vehicle_index]['Graph list']
    G_total_dict[vehicle_id]=G_list
    indexed_target_single=database_dic[vehicle_index]['Indexed Targets']
    targets_dict[vehicle_id]=indexed_target_single
    vehicle_ids.append(vehicle_id)
net = sumolib.net.readNet("D:\\SUMO\\work\\rebuild2\\sourceCICT.net.xml")
control_dict={}
control_dict=control_dictionary_initialize(G_total_dict,targets_dict,vehicle_ids)
edge_search_dic=lane_set_generation(net)
# print(control_dict['49']['uk'])
# print(control_dict['49']['Route'])
# print(control_dict['49']['recorded route'])
# print(control_dict['49']['targets'])
# print(control_dict['49']['stage'])
# print(control_dict['49']['action list'][control_dict['49']['stage']])
# print(control_dict['49']['action list'][control_dict['49']['stage']+1])

# 离线字典生成（需要SUMO和TraCI，但无需实时重生成）

In [7]:
def process_lists(A, B):
    # 创建字典，以 A 中的元素为键，B 中的元素为值
    mapping = {}
    for a, b in zip(A, B):
        if a not in mapping:
            mapping[a] = []
        mapping[a].append(b)

    # 创建字典，以 B 中的元素为键，A 中的元素为值
    reverse_mapping = {}
    for a, b in zip(A, B):
        if b not in reverse_mapping:
            reverse_mapping[b] = []
        reverse_mapping[b].append(a)
    return mapping, reverse_mapping

def trafficlight_dic_generation(TL_id):
    total_phases=traci.trafficlight.getCompleteRedYellowGreenDefinition(TL_id)[0].getPhases()
#     total_connections=net.getNode(TL_id).getConnections()
    controlled_links = traci.trafficlight.getControlledLinks(TL_id)
    crossing_duration=[]
    crossing_edges=[]
    crossing_lanes=[]
    moving_to_lanes=[]
    moving_to_edges=[]
    for i, phase_now in enumerate(total_phases):
        whole_states=phase_now.state
#         print(f'States number: {len(whole_states)}, links number: {len(controlled_links)}')
        duration_now=phase_now.duration
        target_edges=set()
        allowed_edges=set()
        allowed_lanes=[]
        target_lanes=[]
        for j,state_now in enumerate(whole_states):
            if state_now in ['g','G']:
                link_now=controlled_links[j]
                for item in link_now:
                    From_lane, To_lane, _ = item
                FromEdge_id=From_lane.split("_")[0]
                ToEdge_id=To_lane.split("_")[0]
            else:
                continue
            allowed_edges.add(FromEdge_id)
            target_edges.add(ToEdge_id)
            allowed_lanes.append(From_lane)
            target_lanes.append(To_lane)
        mapping_from,mapping_to=process_lists(allowed_lanes, target_lanes)
        crossing_duration.append(duration_now)
        crossing_edges.append(list(allowed_edges))
        crossing_lanes.append(mapping_from)
        moving_to_lanes.append(mapping_to)
        moving_to_edges.append(list(target_edges))
        cycle_time=sum(crossing_duration)
    return TL_id,{'Cycle time':cycle_time,'Phase number': len(total_phases),'Duration list':crossing_duration,'FromEdge list':crossing_edges,'ToEdge list':moving_to_edges,'FromLane list':crossing_lanes,'ToLane list':moving_to_lanes}



# 车道分布预测

In [8]:

def sample_based_graph_generation(edge_id,net):
    G_current=nx.DiGraph()
    if edge_id=='10-11':
        lane_set=[0,1,2]
        vehicle_ids = traci.edge.getLastStepVehicleIDs(edge_id)+traci.edge.getLastStepVehicleIDs(':10_8')
    elif edge_id=='11-10':
        lane_set=[0,1,2]
        vehicle_ids = traci.edge.getLastStepVehicleIDs(edge_id)+traci.edge.getLastStepVehicleIDs(':11_0')
    else:
        edge=net.getEdge(edge_id)
        lanes=edge.getLanes()
        lane_set=[]
        for i in lanes:
            lane_id=i.getID()
            index=int(lane_id.split("_")[-1])
            lane_set.append(index)
        vehicle_ids = traci.edge.getLastStepVehicleIDs(edge_id)
    
    for vehicle in vehicle_ids:
        lane_id=traci.vehicle.getLaneID(vehicle)
        if edge_id=='10-11' or edge_id=='11-10':
            reading_lane_index=int(traci.vehicle.getLaneIndex(vehicle))
            if reading_lane_index==0:
                lane_index=0
            elif reading_lane_index==1 or reading_lane_index==2:
                lane_index=1
            else:
                lane_index=2
        else:
            lane_index=int(lane_id.split("_")[-1])
        s_total=traci.lane.getLength(lane_id)
        s_current=traci.vehicle.getLanePosition(vehicle)
        velocity_current=traci.vehicle.getSpeed(vehicle)
        G_current.add_node(vehicle,Clane=lane_index,speed=velocity_current,position=s_total-s_current)
#     G_current=target_lane_initialize(G_current,lane_set)
    final_nodes_list=[]
    for lane_index in list(sorted(lane_set)):    
        vehicle_in_lane = sorted([node for node, attrs in G_current.nodes(data=True) if attrs.get('Clane') == lane_index],key=lambda node: G_current.nodes[node]['position'])
        for i in range(len(vehicle_in_lane) - 1):
            source = vehicle_in_lane[i]
            target = vehicle_in_lane[i + 1]
            G_current.add_edge(source, target,distance=G_current.nodes[target]['position']-G_current.nodes[source]['position'] )
        if len(vehicle_in_lane)==1:
            final_nodes_list.append(vehicle_in_lane[0])
        elif len(vehicle_in_lane)==0:
            final_nodes_list.append(None)
        else:
            final_nodes_list.append(vehicle_in_lane[i + 1])
                
    return G_current,lane_set,final_nodes_list
            

def group_nodes_by_position(G, judgement_name,target_name,interval=10,down_index=False):
    """
    将图G中的节点按照position属性分组，每组的position值范围相差指定的间隔（默认为10）。
    返回值是一个二维列表，每个子列表包含一个区间内的节点。

    参数:
        G (networkx.Graph): 输入的图，节点必须有position属性。
        interval (int): 每个分组的区间范围，默认为10。

    返回:
        list: 二维列表，每个子列表包含一个区间内的节点。
    """
    # 提取所有节点的position属性
    positions = [data[judgement_name] for _, data in G.nodes(data=True)]

    # 找到position的最大值和最小值
    min_position = min(positions)
    max_position = max(positions)

    # 初始化二维列表
    grouped_nodes = []
    grouped_target=[]

    # 遍历所有节点，计算其对应的区间并分配到分组
    for node, data in G.nodes(data=True):
        position = data[judgement_name]
        # 计算当前节点所属的区间起点
        start = (position // interval) * interval
        # 确保区间起点在合法范围内
        start = max(min_position, min(start, max_position))
        # 构造区间索引
        index = int((start - min_position) // interval)
        # 如果索引超出当前列表范围，动态扩展列表
        while len(grouped_nodes) <= index:
            grouped_nodes.append([])
            grouped_target.append([])
        # 将节点添加到对应的区间
        grouped_nodes[index].append(node)
        grouped_target[index].append(G.nodes[node][target_name])
    if down_index:
        grouped_nodes.reverse()
        grouped_target.reverse()

    return grouped_nodes,grouped_target
    

def target_lane_initialize(G,lane_set):
    if len(G.nodes())==0:
        return G
    G_current=G.copy()
# 考虑车道分配优化问题的目标车道预测
    grouped_nodes,grouped_targets=group_nodes_by_position(G_current, 'position','Clane', 10)
    obstacle_position=[]
    obstacle_target=[]
    for i in range(len(grouped_nodes)):
        assign_position=grouped_targets[i]
        if isinstance(lane_set, list):
            lane_list=lane_set
        else:
            lane_list=list(lane_set)
        available_target = [lane_list.copy() for _ in range(len(assign_position))]
#         print(available_target)
        assigned_lane=lane_assignment_optimization(assign_position,obstacle_position,obstacle_target,available_target,available_target)
#         print(f"assigned lane: {assigned_lane}, based on obstacles: {obstacle_position} to {obstacle_target}, begin with {assign_position}")
        obstacle_position=obstacle_position+assign_position
        obstacle_target=obstacle_target+assigned_lane
        for j in range(len(grouped_nodes[i])):
            node_id=grouped_nodes[i][j]
            mean = assigned_lane[j]
            std_dev = 1
            new_lane_index = np.random.normal(loc=mean, scale=std_dev)
            # 确保新的车道编号在有效范围内 [0, 1, 2]
            new_lane_index = np.clip(new_lane_index, min(lane_set), max(lane_set)).astype(int)
            G_current.nodes[node_id]['Tlane'] = new_lane_index
    return G_current



def holding_time_axis(TL_id,DealedTL_dic,T_total):
#     print(f"traffic light {TL_id} predicting begin--from 'def holding_time_axis'")
    current_phase=traci.trafficlight.getPhase(TL_id)
    abs_next_switching_time=traci.trafficlight.getNextSwitch(TL_id)
    current_time = traci.simulation.getTime()
    next_switching_time=abs_next_switching_time-current_time
#     print(f"time of next switching:{next_switching_time} in {TL_id}--from 'def holding_time_axis'")
    phase_num=DealedTL_dic[TL_id]['Phase number']
    duration_list=DealedTL_dic[TL_id]['Duration list']
    cycle_time=sum(duration_list)
    starting_time=[0]
    ending_time=[next_switching_time]
    holding_phase=[current_phase]
    predict_phase=current_phase
    predict_start=0
    predict_end=T_total
    predict_time=0
    while predict_time<=T_total:
#         print(f'current phase: {holding_phase}')
        predict_phase=holding_phase[-1]+1
        if predict_phase>=phase_num:
            predict_phase=predict_phase-phase_num
        predict_start=ending_time[-1]
        predict_end=ending_time[-1]+duration_list[predict_phase]
        holding_phase.append(predict_phase)
        starting_time.append(predict_start)
        ending_time.append(predict_end)
        predict_time=predict_end
    ending_time[-1]=T_total
#     print(f"traffic light {TL_id} predicting finish--from 'def holding_time_axis'")
    return TL_id,{'Holding phase':holding_phase,'Starting point':starting_time,'Ending point':ending_time}
    
def GNN_prediction(G,T_max=10000,v_ref=15):
    nodes_with_A = [(node, G.nodes[node]['position']) for node in G.nodes if 'position' in G.nodes[node]]
    # 根据属性 A 对节点进行排序（从小到大）
    num_Tlane={}
    num_Clane={}
    node_index_Tlane={}
    node_index_Clane={}
    Clanes_set=set()
    Tlanes_set=set()
    sorted_nodes = sorted(nodes_with_A, key=lambda x: x[1])
    sorted_node_ids = [node[0] for node in sorted_nodes]
    L_traveling=G.nodes[sorted_node_ids[-1]]['position']
    k_distance=0.02
    k_velocity=0.5
    k_theta=1
    k_lateral=0.1
    d_ref=2
    d_timeheadway=0.2
    T_step=0.1
    T_lanechange=5
    lane_width=3
    theta_steer=0.7
    a_max=3
    omega_max=0.5
    state_vehicle_all={}
    if T_max<1:
        print(f"For graph {G.nodes()}, the set prediction horizon {T_max} is too short——from 'def GNN_prediction'")
    for timestamp in range(0,T_max):
        crossing_flag_set=set()
        for node in sorted_node_ids:
            if node not in state_vehicle_all:
                state_vehicle_all[node]=[]
                if G.nodes[node]['Clane'] not in Clanes_set:
                    Clanes_set.add(G.nodes[node]['Clane'])
                    num_Clane[G.nodes[node]['Clane']]=1
                    node_index_Clane[G.nodes[node]['Clane']]=[]
                    node_index_Clane[G.nodes[node]['Clane']].append(node)
                else:
                    num_Clane[G.nodes[node]['Clane']]=1+num_Clane[G.nodes[node]['Clane']]
                    node_index_Clane[G.nodes[node]['Clane']].append(node)
                if G.nodes[node]['Tlane'] not in Tlanes_set:
                    Tlanes_set.add(G.nodes[node]['Tlane'])
                    num_Tlane[G.nodes[node]['Tlane']]=1
                    node_index_Tlane[G.nodes[node]['Tlane']]=[]
                    node_index_Tlane[G.nodes[node]['Tlane']].append(node)
                else:
                    num_Tlane[G.nodes[node]['Tlane']]=1+num_Tlane[G.nodes[node]['Tlane']]
                    node_index_Tlane[G.nodes[node]['Tlane']].append(node)
            if timestamp==0:
                s_current=-G.nodes[node]['position']
                t_current=0
                theta_current=0
                v_current=G.nodes[node]['speed']
            else:
                s_current=state_vehicle_all[node][timestamp-1][0]
                t_current=state_vehicle_all[node][timestamp-1][1]
                theta_current=state_vehicle_all[node][timestamp-1][2]
                v_current=state_vehicle_all[node][timestamp-1][3]
            # 排队加速
            index_current_Clane=node_index_Clane[G.nodes[node]['Clane']].index(node)
            index_current_Tlane=node_index_Tlane[G.nodes[node]['Tlane']].index(node)
            
            if index_current_Clane==0 and index_current_Tlane==0:
                a_k=k_velocity*(v_ref-v_current)
            elif index_current_Clane==0:
                prenode_Tlane=node_index_Tlane[G.nodes[node]['Tlane']][index_current_Tlane-1]
                vpre_Tlane=state_vehicle_all[prenode_Tlane][timestamp-1][3]
                dpre_Tlane=state_vehicle_all[prenode_Tlane][timestamp-1][0]-s_current
                a_k_Clane=k_velocity*(v_ref-v_current)
                a_k_Tlane=k_velocity*(vpre_Tlane-v_current)+k_distance*(dpre_Tlane-d_ref-d_timeheadway*v_current)
                a_k=0.5*a_k_Clane+0.5*a_k_Tlane
            elif index_current_Tlane==0:
                prenode_Clane=node_index_Clane[G.nodes[node]['Clane']][index_current_Clane-1]
                vpre_Clane=state_vehicle_all[prenode_Clane][timestamp-1][3]
                dpre_Clane=state_vehicle_all[prenode_Clane][timestamp-1][0]-s_current
                a_k_Tlane=k_velocity*(v_ref-v_current)
                a_k_Clane=k_velocity*(vpre_Clane-v_current)+k_distance*(dpre_Clane-d_ref-d_timeheadway*v_current)
                a_k=0.5*a_k_Clane+0.5*a_k_Tlane
            else:
                prenode_Clane=node_index_Clane[G.nodes[node]['Clane']][index_current_Clane-1]
                vpre_Clane=state_vehicle_all[prenode_Clane][timestamp-1][3]
                dpre_Clane=state_vehicle_all[prenode_Clane][timestamp-1][0]-s_current
                prenode_Tlane=node_index_Tlane[G.nodes[node]['Tlane']][index_current_Tlane-1]
                vpre_Tlane=state_vehicle_all[prenode_Tlane][timestamp-1][3]
                dpre_Tlane=state_vehicle_all[prenode_Tlane][timestamp-1][0]-s_current
                a_k_Clane=k_velocity*(vpre_Clane-v_current)+k_distance*(dpre_Clane-d_ref-d_timeheadway*v_current)
                a_k_Tlane=k_velocity*(vpre_Tlane-v_current)+k_distance*(dpre_Tlane-d_ref-d_timeheadway*v_current)
                a_k=0.5*a_k_Clane+0.5*a_k_Tlane
            # 换道规划
            lanechange_direction=np.sign(G.nodes[node]['Tlane']-G.nodes[node]['Clane'])
            if lanechange_direction==0:
                omega_k=0
            else:
                if abs(t_current)<abs(G.nodes[node]['Tlane']-G.nodes[node]['Clane'])*lane_width/2:
                    omega_k=k_theta*(theta_steer-theta_current)*lanechange_direction
                else:
                    omega_k=k_theta*(0-theta_current)+k_lateral*((G.nodes[node]['Tlane']-G.nodes[node]['Clane'])*lane_width-t_current)
#             print(f"For vehicle {node} in {G.nodes()} calculated control {[a_k,omega_k]}——from 'def GNN_prediction'")
            a_k=min(abs(a_k),a_max)*np.sign(a_k)
            omega_k=min(abs(omega_k),omega_max)*np.sign(omega_k)
            # 状态空间迭代
            v_next=max(v_current+a_k*T_step,0)
            theta_next=theta_current+omega_k*T_step
            t_next=v_current*np.sin(theta_current)*T_step+t_current
            s_next=v_current*np.cos(theta_current)*T_step+s_current
            state_vehicle_single=[s_next,t_next,theta_next,v_next,a_k,omega_k]
            state_vehicle_all[node].append(state_vehicle_single)
            if s_next>0: # 车辆node穿越路段则植入True
                crossing_flag_set.add(True)
            else: # 车辆node未穿越路段则植入False
                crossing_flag_set.add(False)
#             print(f"vehicle {node} state {state_vehicle_single} at {timestamp}th prediction——from 'def GNN_prediction'")
        if False not in crossing_flag_set: # 所有车辆都穿越了路段
            break  
#     print(f"For graph {G.nodes()} final node {sorted_node_ids[-1]} will travel {L_traveling}m after {timestamp*T_step}s through prediction——from 'def GNN_prediction'")
    if timestamp>9000:
        time.sleep(3)
        print(f"For graph {G.nodes()} target lane queue {node_index_Tlane} current lane queue {node_index_Clane} predicting too long——from 'def GNN_prediction'")
        time.sleep(3)
        for print_i in state_vehicle_all:   
            print(f"vehicle {print_i} predicting {state_vehicle_all[print_i]}——from 'def GNN_prediction'")
            time.sleep(3)

    return max(timestamp,1),state_vehicle_all

def find_best_matching_graph(G0, T0, error_threshold=1):
    """
    使用二分法逼近与目标时间 T0 匹配最佳的子图 G1。
    
    参数:
        G0 (networkx.DiGraph): 完整的有向图。
        T0 (float): 目标时间。
        black_box_function (callable): 黑盒函数，输入为有向图，输出为时间 T。
    
    返回:
        networkx.DiGraph: 与目标时间 T0 匹配最佳的子图 G1。
    """
    # 按照节点的 'position' 属性由大到小排序
    sorted_nodes = sorted(G0.nodes(data=True), key=lambda x: x[1]['position'], reverse=True)
    
    left = 0
    original_right=len(sorted_nodes) - 1
    right = original_right
#   GNN估计通行平均车速
#     speed_best = GNN_prediction(G0)
#     _,dic_selected_initial=sorted_nodes[0]
#     L0_initial=dic_selected_initial['position']
#     T_total=L0_initial/speed_best
#       GNN估计通行时间
    T_total,state_fitting_best=GNN_prediction(G0)
    error=abs(T_total-T0)
    error_last=float('inf')
    G_best=G0.copy()
#     print(error_last >= error)
    counter=0
    while error_last-error>0.001 and error>=error_threshold:
        error_last=error
        mid = (left + right) // 2
        # 从 G0 中删除前 mid 个节点获得G1
        G1 = G0.copy()
        for node, _ in sorted_nodes[:mid]:
            G1.remove_node(node)
        # 计算 G1 的时间 T：基于GNN平均车速
#         _,dic_selected=sorted_nodes[min(mid+1,original_right)]
#         L0=dic_selected['position']
#         speed=GNN_prediction(G1)
#         T = L0 / speed
        # 计算 G1 的时间 T：基于GNN通行时间
        T,state_fitting = GNN_prediction(G1)
#         print(f'current time {T} with left {left}, right {right} and mid {mid}')
        # 计算与 T0 的差值
        error = abs(T - T0)
        if error_last > error:
            G_best=G1.copy()
            state_fitting_best=state_fitting
        # 二分法更新
        if T>T0:
            left = mid
        else:
            right = mid
        if error_last==error:
            counter=counter+1
        if counter>10:
            break

#         print(f'Current error: {error}, Last error: {error_last}, Threshold: {error_threshold}')
    
    return G_best,state_fitting_best    
    
def self_predict_update(duration_time,updating_type,G_original,lane_set,final_nodes):
    if len(G_original.nodes())==0 or duration_time<0.1:
#         print(f"for graph {G_original.nodes()} with duration {duration_time} prediction is non-necessary——from 'def self_predict_update'")
        return G_original,final_nodes
    # 节点属性更新
    lane_set=set()
    if updating_type==0:# 排队状态下
        _,state_predicting=GNN_prediction(G_original,round(duration_time/0.1),0)# 首排期望车速为0，为排队状态
    else:# 通行状态
        _,state_predicting=GNN_prediction(G_original,round(duration_time/0.1))
    for node in G_original.nodes():
        new_position =  -state_predicting[node][-1][0]
        G_original.nodes[node]['position']=max(new_position,0.1)
        lateral_pos=state_predicting[node][-1][1]
        new_lane=round(lateral_pos/3)+G_original.nodes[node]['Clane'] # 以3为车道宽度估计车辆所在车道
        if min(G_original.nodes[node]['Clane'],G_original.nodes[node]['Tlane'])<=new_lane<=max(G_original.nodes[node]['Clane'],G_original.nodes[node]['Tlane']):
            G_original.nodes[node]['Clane']=new_lane
        else:
            G_original.nodes[node]['Clane']=np.random.choice([G_original.nodes[node]['Clane'],G_original.nodes[node]['Tlane'],G_original.nodes[node]['Tlane']])
        new_speed=state_predicting[node][-1][3]
        G_original.nodes[node]['speed']=new_speed
        lane_set.add(G_original.nodes[node]['Clane'])
    # 边更新
    G_copy=G_original.copy()
    G_original.remove_edges_from(G_copy.edges())
    final_nodes_updated=final_nodes
    for lane_index in list(sorted(lane_set)):    
        vehicle_in_lane = sorted([node for node, attrs in G_original.nodes(data=True) if attrs.get('Clane') == lane_index],key=lambda node: G_original.nodes[node]['position'])
        for i in range(len(vehicle_in_lane) - 1):
            source = vehicle_in_lane[i]
            target = vehicle_in_lane[i + 1]
            G_original.add_edge(source, target,distance=G_original.nodes[target]['position']-G_original.nodes[source]['position'] )
            final_nodes_updated[lane_index]=vehicle_in_lane[i + 1]
    return G_original,final_nodes_updated


def lane_distribution_prediction(possible_lane_dic,current_edge,net,DealedTL_dic,edge_search_dic,predicting_time,known_graph_dic={},known_edge_list=[]):
    current_time=traci.simulation.getTime()
    junction_id=current_edge.split("-")[0]
    terminal_junction=current_edge.split("-")[1]
    G_current_initial,lane_current,final_nodes_list=sample_based_graph_generation(current_edge,net)
    G_current_complete=target_lane_initialize(G_current_initial,lane_current)# 初始化目标车道
#     print(f"final nodes:{final_nodes_list}--from 'def lane_distribution_prediction'")
    # 流出预测
    G_predict_outgoing=G_current_complete.copy()
    # 判定路口控制类别，计算预测域内的通行状态
    if net.getNode(terminal_junction).getType()=='traffic_light':
        T_queue=[]
        state_flag=[]
        _,holding_axis=holding_time_axis(terminal_junction,DealedTL_dic,predicting_time)
        for index in range(len(holding_axis['Holding phase'])):
            phase=holding_axis['Holding phase'][index]
            duration=holding_axis['Ending point'][index]-holding_axis['Starting point'][index]
            T_queue.append(duration)
            if current_edge in DealedTL_dic[terminal_junction]['FromEdge list'][phase]:
                state_flag.append(1)
            else:
                state_flag.append(0)   
    else:
        T_queue=[predicting_time]
        state_flag=[1]
    # 根据预测域内的通行状态更新当前路段车辆分布图
    for i in range(len(T_queue)):
        G_predict_outgoing,final_nodes_updated=self_predict_update(T_queue[i],state_flag[i],G_predict_outgoing,lane_current,final_nodes_list)
 
    # 流入预测
    G_predict_incoming=G_predict_outgoing.copy()
    incoming_edges=net.getEdge(current_edge).getIncoming()
    incoming_num=len(incoming_edges)
#     print(f"Incoming edge number: {incoming_num}--from 'def lane_distribution_prediction'")
    edge_list=[]
    G_incoming_list=[]
    final_nodes_list=[]
    lanes_incoming_list=[]
    state_flag_total=[]
    for edge_ic in incoming_edges:
        single_edge_ID=edge_ic.getID()
        G_incoming_initial,lane_incoming,final_incoming=sample_based_graph_generation(single_edge_ID,net)
        if len(G_incoming_initial.nodes())==0:
#             print(f"Empty incoming edge: {single_edge_ID}--from 'def lane_distribution_prediction'")
            incoming_num=incoming_num-1
            continue
        if single_edge_ID in known_edge_list:
            G_incoming_complete=known_graph_dic[single_edge_ID]
            if len(G_incoming_complete.nodes())==0:
                incoming_num=incoming_num-1
                continue
        else:
            G_incoming_complete=target_lane_initialize(G_incoming_initial,lane_incoming)# 初始化目标车道
        edge_list.append(single_edge_ID)
        state_flag_total.append([])
        final_nodes_list.append(final_incoming)
        G_incoming_list.append(G_incoming_complete)
        lanes_incoming_list.append(lane_incoming)
        
    if net.getNode(junction_id).getType()=='traffic_light':
        T_queue=[]
        state_flag=[]
        _,holding_axis=holding_time_axis(junction_id,DealedTL_dic,predicting_time)
        for index in range(len(holding_axis['Holding phase'])):
            phase=holding_axis['Holding phase'][index]
            duration=holding_axis['Ending point'][index]-holding_axis['Starting point'][index]
            T_queue.append(duration)
            if current_edge in DealedTL_dic[junction_id]['ToEdge list'][phase]: # 至少存在一个edge_list[j]到current_edge的流入
                for j in range(incoming_num):
                    if edge_list[j] in DealedTL_dic[junction_id]['FromEdge list'][phase]:
                        state_flag_total[j].append(2) # 从edge_list[j]到current_edge的流入阶段
                    else:
                        state_flag_total[j].append(1) # edge_list[j]不流入current_edge的阶段   
            else: # 不存在任何edge_list[j]到current_edge的流入
                for j in range(incoming_num):
                    if edge_list[j] in DealedTL_dic[junction_id]['FromEdge list'][phase]: # 存在edge_list[j]互相之间的流入
                        state_flag_total[j].append(1) # 由于路口交通规则的限制，edge_list[j]路段的流入并不会影响edge_list[j]路段的流出
                    else:
                        state_flag_total[j].append(0) # 红灯等待  
    else:
        T_queue=[predicting_time]
        for j in range(incoming_num):
            state_flag_total[j].append(2)
    # 根据预测域内的通行状态更新当前路段车辆分布图
#     print(f"Incoming edges {edge_list}, predicted states {state_flag_total}--from 'def lane_distribution_prediction'")
#     print(f" junction {junction_id} {len(T_queue)} phases {T_queue} predicted within horizon {predicting_time} with {incoming_num} incoming at {time.time()}--from 'def lane_distribution_prediction'")
    for i in range(len(T_queue)):
        if T_queue[i]==0:
            continue
        for j in range(incoming_num):
            if 2 not in state_flag_total[j][i:]: # 若后续不再存在到current_edge的流入阶段，则跳过对edge_list[j]路段的分布预测
                continue
#             print(f"incoming {edge_list[j]} predicting with nodes {G_incoming_list[j].nodes()} at {time.time()}--from 'def lane_distribution_prediction'")
            if len(G_incoming_list[j].nodes())==0: # 若edge_list[j]已经滚动预测为空路段，则跳过
                continue
            if state_flag_total[j][i]==2:
#                 print(f"Incoming prediction of {edge_list[j]} begins--from 'def lane_distribution_prediction'")
                G_dynamic,state_fitting=find_best_matching_graph(G_incoming_list[j], T_queue[i])
                crossing_map=edge_search_dic['Possible to lanes'][edge_list[j]][current_edge]
#                 crossing_map=DealedTL_dic[junction_id]['FromLane list'][crossing_phase]# 以incoming车道ID为key，outgoing车道ID列表为value
                # 从小到大排列
                sorted_nodes = sorted(G_dynamic.nodes(data=True), key=lambda x: x[1]['position'], reverse=False)
                for node,_ in sorted_nodes:
                    incoming_lane=G_dynamic.nodes[node]['Tlane']
                    if incoming_lane not in crossing_map:
                        continue
                    totoal_to_lanes=list(crossing_map[incoming_lane]) # set变量需要转为list变量
                    to_lane=random.choice(totoal_to_lanes)
                    int_lane_index=int(to_lane)
#                     print(f"Predicted lane: {int_lane_index} in {lane_current}, where final vehicles are {final_nodes_updated}--from 'def lane_distribution_prediction'")
                    final_node_located=final_nodes_updated[int_lane_index]
                    max_s=net.getEdge(current_edge).getLength()
                    last_position=G_dynamic.nodes[node]['position']
                    traveling_distance=state_fitting[node][-1][0]# 读取预测的沿路坐标
                    if final_node_located==None: # 驶入车道没有前车
                        new_position = np.clip(traveling_distance, 0.1, max_s-0.1).astype(float)
                        G_predict_incoming.add_node(node,Clane=int_lane_index,speed=state_fitting[node][-1][3],position=max_s-new_position)
                        final_nodes_updated[int_lane_index]=node
#                             G_current.add_node(vehicle,Clane=lane_index,speed=velocity_current,position=s_total-s_current)
#                             G_current.add_edge(source, target,distance=G_current.nodes[target]['position']-G_current.nodes[source]['position'] )
                    else:# 驶入车道有前车
                        front_position=G_predict_incoming.nodes[final_node_located]['position'] # 已经过流出预测
                        total_T_predicting=min(len(state_fitting[node]),max(round(round(T_queue[i]/0.1)-1),0))
                        if total_T_predicting==0:
                            print(f"vehicle {node} in fitted graph {G_dynamic.nodes()} incoming within {T_queue[i]} is too fast——from 'def lane_distribution_prediction'")
                            index_i=0
                            speed_located=G_dynamic.nodes[node]['speed']
                            s_located=0
                        else:
                            
                            for index_i in range(0,total_T_predicting):
                                if state_fitting[node][index_i][0]>0:
                                    break
                            speed_located=state_fitting[node][index_i][3]
                            s_located=state_fitting[node][index_i][0]
                        distance_pre=max_s-front_position
                        speed_pre=G_predict_incoming.nodes[final_node_located]['speed']
                        distance_located=min(max(s_located,0.1),distance_pre-0.1)
                        s_updated=distance_located
                        v_updated=speed_located
                        for _ in range(round(T_queue[i]/0.1)-index_i):
                            a_k=0.5*(speed_pre-v_updated)+0.02*(distance_pre-s_updated-5-0.2*v_updated)
                            v_updated=v_updated+a_k*0.1
                            v_updated=max(v_updated,0)
                            s_updated=s_updated+v_updated*0.1
                        new_position = np.clip(max_s-s_updated, front_position+0.1,max_s-0.1 ).astype(float)
                        G_predict_incoming.add_node(node,Clane=int_lane_index,speed=v_updated,position=new_position)
                        source=final_nodes_updated[int_lane_index]
                        target=node
                        G_predict_incoming.add_edge(source, target,distance=G_predict_incoming.nodes[target]['position']-G_predict_incoming.nodes[source]['position'] )
                        final_nodes_updated[int_lane_index]=node
                    G_predict_incoming=target_lane_initialize(G_predict_incoming,lane_current)
                G_incoming_list[j],final_nodes_list[j]=self_predict_update(T_queue[i],1,G_incoming_list[j],lanes_incoming_list[j],final_nodes_list[j])
            else:
                G_incoming_list[j],final_nodes_list[j]=self_predict_update(T_queue[i],state_flag_total[j][i],G_incoming_list[j],lanes_incoming_list[j],final_nodes_list[j])
    return G_predict_incoming,G_predict_outgoing,final_nodes_updated,lane_current


# 基于模型预测控制的车辆实时调度（保证每次采样的动作不一致直至达到要求采样数，总采样次数大于等于设定值）

In [9]:
def random_sample_action_combinations(N, actions, m):
    """
    随机采样动作组合，直到采样数达到最大值m。

    参数:
        N (int): 智能体的数量。
        actions (list of lists): 每个智能体的动作空间，actions[i] 是第 i 个智能体的动作列表。
        m (int): 最大的采样数。

    返回:
        list of lists: 采样的所有动作组合。
    """
    # 生成所有可能的动作组合
    n_max=1
    for i in actions:
        n_max=n_max*len(i)
#     print(f"maximum action number:{n_max}")
    
    # 如果m大于所有可能的组合数，则直接返回所有组合
    
    if m >= n_max:
        all_combination=itertools.product(*actions)
        return [list(comb) for comb in all_combination],1
    else:
        sampled_action_set=set()
        sample_counter=0
        while len(sampled_action_set)<m:
            sample_counter=sample_counter+1
            sampled_action=tuple(random.choice(actions[i]) for i in range(N))
#             print(f"sampled: {sampled_action}")
            sampled_action_set.add(sampled_action)
#         print(f"sample times: {sample_counter}")
        return [list(comb) for comb in sampled_action_set],sample_counter

def two_step_MPC_prediction(net,DealedTL_dic,initialized_dic,control_dict,edge_search_dic,sample_number):
    lane_constrain_dict=edge_search_dic['Possible from lanes']
    possible_lane_dic=edge_search_dic['Possible to lanes']
    current_edge=initialized_dic['Current edge']
    lane_total=initialized_dic['Total lanes']
    G_initial=initialized_dic['Initialized graph']
#     print(G_initial.nodes())
    obstacle_position=initialized_dic['Obstacle current lanes']
    obstacle_target=initialized_dic['Obstacle target lanes']
    vehicle_list=initialized_dic['Decision vehicles']
    position_list=initialized_dic['Vehicle distances']
    _,end_junction_id=current_edge.split("-")
    
    T_preciting_list=[]
    decisions_list=[]
    decision_lanes_list=[]
    predict_decisions_list=[]
    # 预采样节点规划决策
    total_action_space=[]
    total_action_index_space=[]
    for index_preact in range(len(vehicle_list)):
        vehicle_i_preact=vehicle_list[index_preact]
        node_i_preact=current_edge
#         print(f"sample of vehicle {vehicle_i_preact} on edge {node_i_preact} at {time.time()}--from 'def two_step_MPC_prediction'")
        if control_dict[vehicle_i_preact]['uk']==None: # 未决策/决策执行完的车辆，在剪枝后的动作空间中采样
            action_space=control_dict[vehicle_i_preact]['action list'][control_dict[vehicle_i_preact]['stage']][node_i_preact]
        else: # 决策执行中的车辆，第一步为执行中的决策，第二步为下一步采样
            next_decision=control_dict[vehicle_i_preact]['uk']
            next_action_space=control_dict[vehicle_i_preact]['detailed action dictionary'][control_dict[vehicle_i_preact]['stage']][next_decision]
            action_space=[]
            for i in next_action_space:
                action_space.append([next_decision,i])
        total_action_space.append(action_space)
        total_action_index_space.append(list(range(len(action_space))))
    actions_all,_ = random_sample_action_combinations(len(vehicle_list), total_action_index_space, sample_number)
    # 遍历采样结果计算通行时间
    sample_counter=0
    message_step=20
    time_zero=time.time()
    for sampled_action_index in actions_all: # 多次采样
        sample_counter=sample_counter+1
        print(f"!!!{sample_counter}th start at {time.time()-time_zero}!!!--from 'def two_step_MPC_prediction'")
        applied_decision={}
        next_edge_list=[]
        next_next_edge_dic={}
        assign_position=[]
        available_target=[]
        # 采样节点规划决策
        for index in range(len(vehicle_list)):
            vehicle_i=vehicle_list[index]
            node_i=current_edge
            action_sample=total_action_space[index][sampled_action_index[index]]
#             if control_dict[vehicle_i]['uk']==None: # 未决策/决策执行完的车辆，在剪枝后的动作空间中采样
#                 action_space=control_dict[vehicle_i]['action list'][control_dict[vehicle_i]['stage']][node_i]
#                 action_sample=random.choice(action_space)
#             else: # 决策执行中的车辆，第一步为执行中的决策，第二步为下一步采样
#                 next_decision=control_dict[vehicle_i]['uk']
#                 action_sample=[next_decision,random.choice(control_dict[vehicle_i]['detailed action dictionary'][control_dict[vehicle_i]['stage']][next_decision])]
            applied_decision[vehicle_i]=action_sample # 在可能的动作空间中进行多次随机采样
#             print(f"sampled action:{action_sample} of vehicle {vehicle_i} on edge {node_i}--from 'def two_step_MPC_prediction'")
            assign_position.append(G_initial.nodes[vehicle_i]['Clane'])
            available_target.append(lane_constrain_dict[node_i][action_sample[0]])
            next_edge_list.append(action_sample[0])
            next_next_edge_dic[vehicle_i]=action_sample[1]
        if sample_counter % message_step == 1:
            print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
            print(f"vehicle {vehicle_list} firstly initialize target lanes at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
        assigned_lane=lane_assignment_optimization(assign_position,obstacle_position,obstacle_target,available_target,lane_total)
        for index in range(len(vehicle_list)):
            vehicle_i=vehicle_list[index]
            G_initial.nodes[vehicle_i]['Tlane']=assigned_lane[index]
#         print(f"decision graph:{G_initial} with nodes:{G_initial.nodes()}--from 'def two_step_MPC_prediction'")
        T_traveling_1,state_vehicle_dic_1=GNN_prediction(G_initial) # 基于采样的决策进行车道分配规划，计算通行时长
        if sample_counter % message_step == 1:
#             print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
            print(f"first traveling time {T_traveling_1} calculated at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
        # 计算红绿灯路口等待时长
        if net.getNode(end_junction_id).getType()=='traffic_light':
            T_waitting_1=None
            cycle_time=DealedTL_dic[end_junction_id]['Cycle time']
            _,holding_axis=holding_time_axis(end_junction_id,DealedTL_dic,T_traveling_1+cycle_time)
#             print(f"predicting time:{T_traveling_1+cycle_time}--from 'def two_step_MPC_prediction'")
#             print(f"predicted result:{holding_axis}--from 'def two_step_MPC_prediction'")
            for index in range(len(holding_axis['Holding phase'])):
                phase=holding_axis['Holding phase'][index]
                crossing_edge_list=DealedTL_dic[end_junction_id]['FromEdge list'][phase]
#                 print(f"current egde:{current_edge}, crossing list:{crossing_edge_list}--from 'def two_step_MPC_prediction'")
                if current_edge in crossing_edge_list and holding_axis['Ending point'][index]>=T_traveling_1:
                    allowed_TL_time=holding_axis['Starting point'][index]
                    T_waitting_1=max(allowed_TL_time-T_traveling_1,0)
                    break
            if T_waitting_1==None:
                print(f'红绿灯路口{end_junction_id}等待时间计算出错:{current_edge}无法通过！')
        else:
            T_waitting_1=0
        if sample_counter % message_step == 1:
#             print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
            print(f"first waitting time {T_waitting_1} calculated at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
        G_without_decision=G_initial.copy()
        G_without_decision.remove_nodes_from(vehicle_list)
        predict_edges_set=set()
        G_predict_dic={}
        final_nodes_dic={}
        lanes_dic={}
        vehicles_dic={}
        decision_lanes_list_single=[]
        if sample_counter % message_step == 1:
#             print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
            print(f"edges {next_edge_list} prediction begin at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
            
        for index_i in range(len(next_edge_list)):
            next_edge=next_edge_list[index_i]
            max_s=net.getEdge(next_edge).getLength()
            if next_edge in predict_edges_set:
                G_predict=G_predict_dic[next_edge]
                final_nodes_updated=final_nodes_dic[next_edge]
                lane_current=lanes_dic[next_edge]
            else:
#                 print(f"predicted edge:{next_edge}--from 'def two_step_MPC_prediction'")
#                                                                 lane_distribution_prediction(possible_lane_dic,current_edge,net,DealedTL_dic,edge_search_dic,predicting_time,known_graph_dic={},known_edge_list=[])
                
#                 print(f"predicting edge:{next_edge}--from 'def two_step_MPC_prediction'")
                G_predict,_,final_nodes_updated,lane_current=lane_distribution_prediction(possible_lane_dic,next_edge,net,DealedTL_dic,edge_search_dic,T_traveling_1+T_waitting_1,{current_edge:G_without_decision},current_edge)
                vehicles_dic[next_edge]=[]
            ego_vehicle=vehicle_list[index_i]
            possible_lane=possible_lane_dic[current_edge][next_edge][G_initial.nodes[ego_vehicle]['Tlane']] # set变量
#             print(f"possible lanes {possible_lane}--from 'def two_step_MPC_prediction'")
            arrive_lane=random.choice(list(possible_lane))
            traveling_distance=state_vehicle_dic_1[ego_vehicle][-1][0]# 读取建模预测的沿路坐标，必然大于等于0，驶出了当前路段
            current_s=G_initial.nodes[ego_vehicle]['position']
            predict_position=max(max_s-traveling_distance,0.1)
            predict_speed=state_vehicle_dic_1[ego_vehicle][-1][3]# 读取建模预测的车速
#             print(f"for node: {ego_vehicle} Clane:{arrive_lane},speed:{predict_speed},position:{predict_position}--from 'def two_step_MPC_prediction'")    
            G_predict.add_node(ego_vehicle,Clane=arrive_lane,speed=predict_speed,position=predict_position)
            current_final=final_nodes_updated[arrive_lane]
            if current_final==None:
                final_nodes_updated[arrive_lane]=ego_vehicle
            else:
                G_predict.add_edge(current_final, ego_vehicle,distance=G_predict.nodes[ego_vehicle]['position']-G_predict.nodes[current_final]['position'] )
                final_nodes_updated[arrive_lane]=ego_vehicle
            predict_edges_set.add(next_edge)
#             print(f"predicted graph {G_predict}--from 'def two_step_MPC_prediction'")
            G_predict_dic[next_edge]=G_predict
            final_nodes_dic[next_edge]=final_nodes_updated
            lanes_dic[next_edge]=lane_current
            vehicles_dic[next_edge].append(ego_vehicle)
            decision_lanes_list_single.append(arrive_lane)
        if sample_counter % message_step == 1:
#             print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
            print(f"edges {next_edge_list} prediction ended at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
            
            # 在预测的路段车辆分布有向图中添加节点
            # 在预测的路段车辆分布有向图中添加边
        predicting_traveling_time_list=[]
        predicting_waitting_time_list=[]
        for key_index in G_predict_dic:
            assign_position_after=[]
            obstacle_position_after=[]
            obstacle_target_after=[]
            available_target_after=[]
            G_selected=G_predict_dic[key_index]
            lane_total_2=lanes_dic[key_index]
#             print(f"current graph:{G_selected} with nodes:{G_selected.nodes()}--from 'def two_step_MPC_prediction'")
            for node_i in G_selected.nodes():
#                 print(f"node {node_i} with Clane:{G_selected.nodes[node_i]['Clane']}--from 'def two_step_MPC_prediction'")
                if node_i in vehicles_dic[key_index]:
                    assign_position_after.append(G_selected.nodes[node_i]['Clane'])
                    next_next_edge=next_next_edge_dic[node_i]
                    if next_next_edge==None: # 若采样的第一步到达了终点，第二步是None，则约束变为全车道
                        available_target_after.append(lane_total_2)
                    else:
                        available_target_after.append(lane_constrain_dict[key_index][next_next_edge])
                else:
                    obstacle_position_after.append(G_selected.nodes[node_i]['Tlane'])
                    obstacle_target_after.append(G_selected.nodes[node_i]['Tlane'])
#             print(f"assign position:{assign_position_after},lane space:{available_target_after}--from 'def two_step_MPC_prediction'")
#             print(f"obstacle position:{obstacle_position_after},obstacle target:{obstacle_target_after}--from 'def two_step_MPC_prediction'")
            if sample_counter % message_step == 1:
#                     print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")        
                print(f"vehicle {vehicle_list} secondly initialize target lanes on {key_index} at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
            assigned_lane=lane_assignment_optimization(assign_position,obstacle_position,obstacle_target,available_target,lane_total_2)
            for num_index in range(len(vehicles_dic[key_index])):
                ego_vehicle=vehicles_dic[key_index][num_index]
                G_selected.nodes[ego_vehicle]['Tlane']=assigned_lane[num_index]
            T_traveling_2,_=GNN_prediction(G_selected)
            if sample_counter % message_step == 1:
#                 print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
                print(f"second traveling time {T_traveling_2} on {key_index} calculated at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
#             print(f"calculated traveling time: {T_traveling_2}--from 'def two_step_MPC_prediction'")
            end_junction_id_2=key_index.split("-")[1]
            if net.getNode(end_junction_id_2).getType()=='traffic_light':
                T_waitting_2=None
                cycle_time_2=DealedTL_dic[end_junction_id_2]['Cycle time']
                _,holding_axis_2=holding_time_axis(end_junction_id_2,DealedTL_dic,T_traveling_2+cycle_time_2)
#                 print(f"traveling time:{T_traveling_2}, cycle time: {cycle_time_2}--from 'def two_step_MPC_prediction'")
#                 print(f"predicted result:{holding_axis_2}--from 'def two_step_MPC_prediction'")
                for index_2 in range(len(holding_axis_2['Holding phase'])):
                    phase_2=holding_axis_2['Holding phase'][index_2]
                    crossing_edge_list_2=DealedTL_dic[end_junction_id_2]['FromEdge list'][phase_2]
#                     print(f"current egde:{key_index}, crossing list:{crossing_edge_list}--from 'def two_step_MPC_prediction'")
                    if key_index in crossing_edge_list_2 and holding_axis_2['Ending point'][index_2]>=T_traveling_2:
                        allowed_TL_time_2=holding_axis_2['Starting point'][index_2]
                        T_waitting_2=max(allowed_TL_time_2-T_traveling_2,0)
                        break
                if T_waitting_2==None:
                    print(f'红绿灯路口{end_junction_id_2}等待时间计算出错:{key_index}无法通过！')
            else:
                T_waitting_2=0
            if sample_counter % message_step == 1:
#                 print(f"————————{sample_counter}th sample predicting————————--from 'def two_step_MPC_prediction'")
                print(f"second waitting time {T_waitting_2} at {end_junction_id_2} calculated at {time.time()-time_zero}--from 'def two_step_MPC_prediction'")
        if sample_counter % message_step == 1:
            print(f"————————{sample_counter}th sample ended————————--from 'def two_step_MPC_prediction'")
        predicting_traveling_time_list.append(T_traveling_2)
        predicting_waitting_time_list.append(T_traveling_2)
#             print(f"prediction of {sample_counter}th sample succeed--from 'def two_step_MPC_prediction'")
        T_taking_sample=T_traveling_1+T_waitting_1+np.mean(predicting_traveling_time_list)+np.mean(predicting_waitting_time_list)
        T_preciting_list.append(T_taking_sample)
        decisions_list.append(next_edge_list)
        decision_lanes_list.append(decision_lanes_list_single)
        predict_decisions_list.append(next_next_edge_dic)
    return T_preciting_list,decisions_list,decision_lanes_list,predict_decisions_list


def MPC_schedule(net,vehicle_id,control_dic,DealedTL_dic,edge_search_dic,sample_number=100):
    current_lane_full=traci.vehicle.getLaneID(vehicle_id)
#     print(f"full lane ID:{current_lane_full} of vehicle:{vehicle_id}--from 'def MPC_schedule'")
    if current_lane_full.split("_")[0]==':10':
        edge_id='10-11'
        reading_lane_index=int(traci.vehicle.getLaneIndex(vehicle_id))
        if reading_lane_index==0:
            lane_index_src='0'
        elif reading_lane_index==1 or reading_lane_index==2:
            lane_index_src='1'
        else:
            lane_index_src='2'
    elif current_lane_full.split("_")[0]==':11':
        edge_id='11-10'
        reading_lane_index=int(traci.vehicle.getLaneIndex(vehicle_id))
        if reading_lane_index==0:
            lane_index_src='0'
        elif reading_lane_index==1 or reading_lane_index==2:
            lane_index_src='1'
        else:
            lane_index_src='2'
    else:
        edge_id,lane_index_src=current_lane_full.split("_")
    
    G_current_initial,lane_current,_=sample_based_graph_generation(edge_id,net)
    grouped_nodes,grouped_targets=group_nodes_by_position(G_current_initial, 'position','position', 10)
    nodes_to_remain=[]
    for i in range(len(grouped_nodes)):
        selected_group=grouped_nodes[i]
        selected_targets=grouped_targets[i]
        nodes_to_remain=nodes_to_remain+selected_group
        if vehicle_id in selected_group:
            break
    G_remove=G_current_initial.copy()
    G_remove.remove_nodes_from(nodes_to_remain)
    nodes_to_remove=list(G_remove.nodes())
    G_current_initial.remove_nodes_from(nodes_to_remove)
    obstacle_position=[]
    obstacle_target=[]
    for j in range(i+1):
        for node_selected in grouped_nodes[j]:
            assign_position=[G_current_initial.nodes[node_selected]['Clane']]
            if control_dict[node_selected]['Tlane']==None:
#                 print(f'assign position:{assign_position}, lane space:{lane_current}')
                assigned_lane=lane_assignment_optimization(assign_position,obstacle_position,obstacle_target,[lane_current],lane_current)
                G_current_initial.nodes[node_selected]['Tlane']=assigned_lane[0]
            else:
                assigned_lane=control_dict[node_selected]['Tlane']
                G_current_initial.nodes[node_selected]['Tlane']=assigned_lane
            obstacle_position=obstacle_position+assign_position
            obstacle_target=obstacle_target+assigned_lane
    print(f"vehicles:{selected_group} on edge:{edge_id} begin MPC at {time.time()}--from 'def MPC_schedule'")
    initialized_dic={'Decision vehicles':selected_group,'Vehicle distances':selected_targets,'Current edge':edge_id,'Total lanes':lane_current,'Initialized graph':G_current_initial,'Obstacle current lanes':obstacle_position,'Obstacle target lanes':obstacle_target}
    T_preciting_list,decisions_list,decision_lanes_list,predict_decisions_list=two_step_MPC_prediction(net,DealedTL_dic,initialized_dic,control_dic,edge_search_dic,sample_number)
    min_value = min(T_preciting_list)
    min_index = T_preciting_list.index(min_value)
#     print(f"optimal index:{min_index},MPC time:{T_preciting_list},MPC u:{decisions_list},MPC lane:{decision_lanes_list}--from 'def MPC_schedule'")
    optimal_decision=decisions_list[min_index]
    optimal_Tlane=decision_lanes_list[min_index]
    optimal_next_decision=predict_decisions_list[min_index]
    for index in range(len(selected_group)):
        vehicle_current=selected_group[index]
        decision_current=optimal_decision[index]
        predict_decision_current=optimal_next_decision[vehicle_current]
        Tlane_current=optimal_Tlane[index]
        control_dic[vehicle_current]['uk']=decision_current
        control_dic[vehicle_current]['Tlane']=Tlane_current
        if predict_decision_current==None:
            control_dic[vehicle_current]['Route']=[edge_id,decision_current]
        else:
            control_dic[vehicle_current]['Route']=[edge_id,decision_current,predict_decision_current]
#     print(f"optimal decision:{optimal_decision} on {edge_id} of vehicle {selected_group}--from 'def MPC_schedule'")
    vehicle_id_index=selected_group.index(vehicle_id)
    decision_vehicle_id=optimal_decision[vehicle_id_index]
    Tlane_vehicle_id=optimal_Tlane[vehicle_id_index]
    print(f"MPC succeed at {time.time()}--from 'def MPC_schedule'")
    clear_output(wait=True)
    return vehicle_id,decision_vehicle_id,Tlane_vehicle_id
        
    

# SUMO启动脚本

In [22]:
# 关闭TraCI连接
traci.close()
print("SUMO终端已关闭")

SUMO终端已关闭


## DJT/DJL离线baseline仿真与记录

In [ ]:

def initialize_traffic_lights_randomly(tl_id):
    """
    随机初始化指定交通灯的初始相位。
    
    参数:
    tl_id (str): 交通灯的ID
    """
    # 获取交通灯的所有相位信息
    logics = traci.trafficlight.getAllProgramLogics(tl_id)
    logic = logics[0]  # 获取第一个控制方案
    phases = logic.phases  # 获取所有相位
    
    # 随机选择一个相位作为初始相位
    initial_phase_index = random.randint(0, len(phases) - 1)
    traci.trafficlight.setPhase(tl_id, initial_phase_index)
    
    # 输出选择的初始相位信息
    return initial_phase_index
def initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic):
    traci.trafficlight.setPhase(tl_id, tl_initialize_dic[tl_id])
    return tl_initialize_dic[tl_id]
    

def run_simulation(sumocfg_file,tl_initialize_dic,random_flag=True):
    """
    运行SUMO仿真。
    
    参数:
    sumocfg_file (str): SUMO配置文件的路径
    """
    # 启动SUMO仿真
#     traci.start(['sumo-gui', '-c', sumocfg_file])
    traci.start(['sumo-gui', '-c', sumocfg_file, '--step-length', '0.1'])
    # 获取所有交通灯的ID
    tls_ids = traci.trafficlight.getIDList()
    
    # 随机初始化每个交通灯的初始相位
    DealedTL_dic={}
    initialized_traffic_lights={}
    for tl_id in tls_ids:
        if random_flag:
            initialized_traffic_lights[tl_id]=initialize_traffic_lights_randomly(tl_id)
        else:
            initialized_traffic_lights[tl_id]=initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic)
                
    # 运行仿真
    MPC_counter=0
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
#         print(f'Current time: {current_time}')
    
    # 关闭TraCI连接
    traci.close()
    print("SUMO终端已关闭")
    return initialized_traffic_lights

# output_path_config= f"D:\\SUMO\\work\\rebuild2\\InitialPhases{simulating_data_index}.pkl" 
# with open(output_path_config, 'rb') as file:
#     tl_initialize_dic = pickle.load(file)
# print(f"{output_path_config}数据已加载")
# # SUMO配置文件路径
# sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJL.sumocfg"
# # sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJT.sumocfg"
# # sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_mix.sumocfg"
# # sumocfg_file = "D:\\SUMO\\work\\rebuild2\\proposed_DMPC.sumocfg"

# # 运行仿真
# start_time = time.time()
# initialized_traffic_lights=run_simulation(sumocfg_file,tl_initialize_dic,False)#True：随机生成初始红绿灯相位并保存/False：载入已有配置
# end_time = time.time()      
# elapsed_time = end_time - start_time
# output_file_path_light = r"D:\\SUMO\\work\\rebuild2\\InitialPhases_copy.pkl"  # 替换为你想要保存的文件名
# with open(output_file_path_light, 'wb') as file:
#     pickle.dump(initialized_traffic_lights, file)
# print(f"测试耗时：{elapsed_time}秒，数据已保存到：{output_file_path_light}")


import xml.etree.ElementTree as ET
import xlsxwriter
import os
import xml.etree.ElementTree as ET
import xlsxwriter
import os
from scipy.io import savemat

# 读取xml文件，写入excel
def datadeal(file_xml):
    vehicle_indicator={}
    vehicle_dataframe={}
    frame_indicator={}
    vehicle_set=set()
    for event, elem in ET.iterparse(file_xml, events=('end',)):
        if elem.tag == 'timestep':
            time = elem.get('time')
            if float(time)%500==0:
                print(f"processing time {time}")
            vehicle_num_current=0
            vehicle_speed_current=0
            for vehicle in elem.findall('vehicle'):
                vehicle_id = vehicle.get('id')
                if vehicle_id not in vehicle_set:
                    vehicle_set.add(vehicle_id)
                    vehicle_dataframe[f'vehicle_{vehicle_id}']={}
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['time']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['x']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['y']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['speed']=[]
                x_num = vehicle.get('x')
                y_num = vehicle.get('y')
                speed = vehicle.get('speed')
                vehicle_dataframe[f'vehicle_{vehicle_id}']['time'].append(float(time))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['x'].append(float(x_num))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['y'].append(float(y_num))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['speed'].append(float(speed))
                vehicle_num_current=vehicle_num_current+1
                vehicle_speed_current=vehicle_speed_current+float(speed)
            frame_indicator[f'time_{int(float(time)*1000)}_ms']=np.array([vehicle_num_current,vehicle_speed_current/max(vehicle_num_current,1)])
            elem.clear()
    for vehicle_id_ind in vehicle_dataframe:
        start_time=min(vehicle_dataframe[vehicle_id_ind]['time'])
        end_time=max(vehicle_dataframe[vehicle_id_ind]['time'])
        total_speed=sum(vehicle_dataframe[vehicle_id_ind]['speed'])
        x_list=vehicle_dataframe[vehicle_id_ind]['x']
        diff_x=[x_list[i+1] - x_list[i] for i in range(len(x_list) - 1)]
        y_list=vehicle_dataframe[vehicle_id_ind]['x']
        diff_y=[y_list[i+1] - y_list[i] for i in range(len(y_list) - 1)]
        distance_step=[a**2 + b**2 for a, b in zip(diff_x, diff_y)]
        vehicle_indicator[vehicle_id_ind]={}
        vehicle_indicator[vehicle_id_ind]['appear']=start_time
        vehicle_indicator[vehicle_id_ind]['disappear']=end_time
        vehicle_indicator[vehicle_id_ind]['distance']=sum(distance_step)
        vehicle_indicator[vehicle_id_ind]['averaspeed']=sum(distance_step)/(end_time-start_time)
        vehicle_dataframe[vehicle_id_ind]['time']=np.array(vehicle_dataframe[vehicle_id_ind]['time'])
        vehicle_dataframe[vehicle_id_ind]['x']=np.array(vehicle_dataframe[vehicle_id_ind]['x'])
        vehicle_dataframe[vehicle_id_ind]['y']=np.array(vehicle_dataframe[vehicle_id_ind]['y'])
        vehicle_dataframe[vehicle_id_ind]['speed']=np.array(vehicle_dataframe[vehicle_id_ind]['speed'])

    # 关闭文件
    return vehicle_indicator,frame_indicator,vehicle_dataframe

simulating_data_index = 14
output_path_config= f"D:\\SUMO\\work\\rebuild2\\InitialPhases{simulating_data_index}.pkl" 
with open(output_path_config, 'rb') as file:
    tl_initialize_dic = pickle.load(file)
print(f"{output_path_config}数据已加载")
# SUMO配置文件路径
# sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJL.sumocfg"
sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJT.sumocfg"
# 运行仿真
start_time = time.time()
initialized_traffic_lights=run_simulation(sumocfg_file,tl_initialize_dic,False)#True：随机生成初始红绿灯相位并保存/False：载入已有配置
end_time = time.time()      
elapsed_time = end_time - start_time
# 设置文件路径
source_folder = "D:\\SUMO\\work\\rebuild2\\"  # 源文件夹路径
#     xml_file_path = os.path.join(source_folder, 'fcd_DJL.xml')
xml_file_path = os.path.join(source_folder, 'fcd_DJT.xml')


# indexes_select=[7,18]
# # indexes_select=[18]
# full_dealed_data={}
# for simulating_data_index in indexes_select:
#     # 设置文件路径
#     # simulating_data_index = 9
#     output_path_config= f"D:\\SUMO\\work\\rebuild2\\InitialPhases{simulating_data_index}.pkl" 
#     with open(output_path_config, 'rb') as file:
#         tl_initialize_dic = pickle.load(file)
#     print(f"{output_path_config}数据已加载")
#     # SUMO配置文件路径
#     sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJL.sumocfg"
# #     sumocfg_file = "D:\\SUMO\\work\\rebuild2\\baseline_DJT.sumocfg"
#     # 运行仿真
#     start_time = time.time()
#     initialized_traffic_lights=run_simulation(sumocfg_file,tl_initialize_dic,False)#True：随机生成初始红绿灯相位并保存/False：载入已有配置
#     end_time = time.time()      
#     elapsed_time = end_time - start_time
#     # 设置文件路径
#     source_folder = "D:\\SUMO\\work\\rebuild2\\"  # 源文件夹路径
# #     xml_file_path = os.path.join(source_folder, 'fcd_DJL.xml')
#     xml_file_path = os.path.join(source_folder, 'fcd_DJT.xml')
#     # 调用函数
#     print(f"{xml_file_path} converting")
#     vehicle_indicator,frame_indicator,vehicle_dataframe=datadeal(xml_file_path)
#     full_dealed_data[f'file_{simulating_data_index}']={}
#     full_dealed_data[f'file_{simulating_data_index}']['vehicle_indicator']=vehicle_indicator
#     full_dealed_data[f'file_{simulating_data_index}']['frame_indicator']=frame_indicator
#     full_dealed_data[f'file_{simulating_data_index}']['dataframes']=vehicle_dataframe
#     clear_output(wait=True)
#     single_dealed_data={}
#     single_dealed_data['vehicle_indicator']=vehicle_indicator
#     single_dealed_data['frame_indicator']=frame_indicator
#     single_dealed_data['dataframes']=vehicle_dataframe
# #     with open(f"D:\\SUMO\\work\\rebuild2\\DJL_data_{simulating_data_index}.pkl", 'wb') as file:
#     with open(f"D:\\SUMO\\work\\rebuild2\\DJT_data_{simulating_data_index}.pkl", 'wb') as file:
#         pickle.dump(single_dealed_data, file)
#     print(f"测试耗时：{elapsed_time}秒，数据已保存：{xml_file_path}")
    
# # save_path = r"D:\\SUMO\\work\\rebuild2\\DJL_data.pkl"  # 替换为你想要保存的文件名
# save_path = r"D:\\SUMO\\work\\rebuild2\\DJT_data.pkl"  # 替换为你想要保存的文件名
# with open(save_path, 'wb') as file:
#     pickle.dump(full_dealed_data, file)
# print(f"数据已保存到：{save_path}")

## DMPC仿真与记录

In [ ]:
def initialize_traffic_lights_randomly(tl_id):
    """
    随机初始化指定交通灯的初始相位。
    
    参数:
    tl_id (str): 交通灯的ID
    """
    # 获取交通灯的所有相位信息
    logics = traci.trafficlight.getAllProgramLogics(tl_id)
    logic = logics[0]  # 获取第一个控制方案
    phases = logic.phases  # 获取所有相位
    
    # 随机选择一个相位作为初始相位
    initial_phase_index = random.randint(0, len(phases) - 1)
    traci.trafficlight.setPhase(tl_id, initial_phase_index)
    return initial_phase_index

def initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic):
    traci.trafficlight.setPhase(tl_id, tl_initialize_dic[tl_id])
    return tl_initialize_dic[tl_id]

def run_simulation(sumocfg_file,net,control_dic,G_total_dict,targets_dict,edge_search_dic,tl_initialize_dic,random_flag=True,sample_number=100):
    """
    运行SUMO仿真。
    
    参数:
    sumocfg_file (str): SUMO配置文件的路径
    """
    start_time = time.time()
    # 启动SUMO仿真
#     traci.start(['sumo-gui', '-c', sumocfg_file])
    traci.start(['sumo-gui', '-c', sumocfg_file, '--step-length', '0.1'])
    # 获取所有交通灯的ID
    tls_ids = traci.trafficlight.getIDList()
    
    # 随机初始化每个交通灯的初始相位
    DealedTL_dic={}
    for tl_id in tls_ids:
        # 初始化数据字典
        key_tl,value_tl=trafficlight_dic_generation(tl_id)
#         print(f'trafficlight id: {tl_id}, applied id: {key_tl}')
        DealedTL_dic[key_tl]=value_tl
        if random_flag:
            initialize_traffic_lights_randomly(tl_id)
        else:
            initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic)
    # 运行仿真
    step = 0
    MPC_counter=0
#     last_message=None
    edge_id_last={}
    while traci.simulation.getMinExpectedNumber() > 0:
        if step % 1 == 0:
            vehicles_all=traci.vehicle.getIDList()
            # 检查所有车辆是否执行至决策点，重置对应动作
            for vehicle_i_1 in vehicles_all:
                edge_id_1 = traci.vehicle.getRoadID(vehicle_i_1)
#                 if vehicle_i_1=='14' and edge_id_1!=last_message:
#                     last_message=edge_id_1
#                     print(f"at {traci.simulation.getTime()}s vehicle {vehicle_i_1} on {edge_id_1} where route {control_dic[vehicle_i_1]['Route']} and stage {control_dic[vehicle_i_1]['stage']} with targets {targets_dict[vehicle_i_1]}")
                if edge_id_1==':11_0':
                    edge_id_1='11-10'
                elif edge_id_1==':10_8':
                    edge_id_1='10-11'
                
                # 在车辆移动过程中记录实际移动的路线
                if not edge_id_1.startswith(":") and (edge_id_1 not in control_dic[vehicle_i_1]['recorded route'] or edge_id_1!=edge_id_last[vehicle_i_1]):
                    control_dic[vehicle_i_1]['recorded route'].append(edge_id_1)
                    edge_id_last[vehicle_i_1]=edge_id_1
                if edge_id_1 in control_dic[vehicle_i_1]['Route'] and edge_id_1!=control_dic[vehicle_i_1]['Route'][0]:
                    control_dic[vehicle_i_1]['uk']=None
                    control_dic[vehicle_i_1]['Tlane']=None
#                     if edge_id_1==targets_dict[vehicle_i_1][control_dic[vehicle_i_1]['stage']]:
                    if edge_id_1==targets_dict[vehicle_i_1][control_dic[vehicle_i_1]['stage']]:
                        control_dic[vehicle_i_1]['stage']=control_dic[vehicle_i_1]['stage']+1
                        # 若移动至最后一个终点路段，则视为到达终点，在调度过程中忽略该车辆
                        if control_dic[vehicle_i_1]['stage']>=len(targets_dict[vehicle_i_1]):
                            print(f"deleted vehicle { vehicle_i_1} with finished traveling")
                            traci.vehicle.remove(vehicle_i_1)   
                else:
                    continue
            vehicles_all_updated=traci.vehicle.getIDList()
            # MPC求解决策，更新动作
            for vehicle_i_2 in vehicles_all_updated:
                edge_id_2 = traci.vehicle.getRoadID(vehicle_i_2)
                if edge_id_2==':11_0':
                    edge_id_2='11-10'
                elif edge_id_2==':10_8':
                    edge_id_2='10-11'
                if control_dic[vehicle_i_2]['uk']==None and not edge_id_2.startswith(":"):
#                     print(f"vehicle {vehicle_i_2} begins MPC scheduling on {edge_id_2}")
                    vehicle_id_now,decision_now,Tlane_now=MPC_schedule(net,vehicle_i_2,control_dic,DealedTL_dic,edge_search_dic,sample_number)
                    control_dic[vehicle_id_now]['uk']=decision_now
                    control_dic[vehicle_id_now]['Tlane']=Tlane_now
                    MPC_counter=MPC_counter+1
                else:
                    continue
            # 执行决策动作
            for vehicle_i_3 in vehicles_all_updated:
                edge_id_3 = traci.vehicle.getRoadID(vehicle_i_3)
                current_route=list(traci.vehicle.getRoute(vehicle_i_3))
                
#                 print(f"route reading:{current_route}, route writting:{control_dic[vehicle_i_3]['Route']}, comparison: {control_dic[vehicle_i_3]['Route']==current_route} ")
#                 if edge_id_3==':10_8' and '11-12' not in control_dic[vehicle_i_3]['Route']: # 边10-11和11-10极小，无法读取车辆位置，需要特殊处理
#                     control_dic[vehicle_i_3]['Route'].append('11-12')
#                 elif edge_id_3==':11_0':
#                     append_flag=True
#                     avaliable_actions=control_dic[vehicle_i_3]['detailed action dictionary'][control_dict[vehicle_i_3]['stage']]['11-10']
#                     for action_i in avaliable_actions:
#                         if action_i==None:
#                             continue
#                         if action_i in control_dic[vehicle_i_3]['Route']:
#                             append_flag=False
#                             break
#                     if append_flag:
#                         selected_action=random.choice(avaliable_actions)
#                         control_dic[vehicle_i_3]['Route'].append(selected_action)
                if edge_id_3==':11_0':
                    edge_id_3='11-10'
                elif edge_id_3==':10_8':
                    edge_id_3='10-11'
                if control_dic[vehicle_i_3]['Route']==current_route or edge_id_3.startswith(":"):
                    continue
                else:
                    next_edge=control_dic[vehicle_i_3]['uk']
                    instant_route=control_dic[vehicle_i_3]['Route']
#                     print(f"optimized route {instant_route} on {edge_id_3} of vehicle {vehicle_i_3}")
                    traci.vehicle.setRoute(vehicle_i_3, instant_route)
        traci.simulationStep()
        current_time = traci.simulation.getTime()
#         print(f'Current time: {current_time}')
        step += 1
    end_time = time.time()      
    elapsed_time = end_time - start_time
    print(f"耗时{elapsed_time}秒")
    # 关闭TraCI连接
    traci.close()
    print(f"SUMO终端已关闭")
    return MPC_counter

# 选择初始相位数据组
simulating_data_index=2

seleted_indexes_list_global=database_baseline['Routes index list']
# print(seleted_indexes_list)
G_total_dict_global={}
targets_dict_global={}
vehicle_ids_global=[]
for iteration_index_global in range(len(seleted_indexes_list_global)):
    vehicle_index_global=seleted_indexes_list_global[iteration_index_global]
    vehicle_id_global=str(iteration_index_global)
    current_data_index_global=seleted_indexes_list_global[vehicle_index_global]
    G_list_global=database_dic[vehicle_index_global]['Graph list']
    G_total_dict_global[vehicle_id_global]=G_list_global
    indexed_target_single_global=database_dic[vehicle_index_global]['Indexed Targets']
    targets_dict_global[vehicle_id_global]=indexed_target_single_global
    vehicle_ids_global.append(vehicle_id_global)  
net_global = sumolib.net.readNet("D:\\SUMO\\work\\rebuild2\\sourceCICT.net.xml")
control_dic_global=control_dictionary_initialize(G_total_dict_global,targets_dict_global,vehicle_ids_global)
edge_search_dic_global=lane_set_generation(net_global)
# 交通灯初始配置
# output_path_config_global= r"D:\\SUMO\\work\\rebuild2\\InitialPhases2.pkl" 
output_path_config_global= f"D:\\SUMO\\work\\rebuild2\\InitialPhases{simulating_data_index}.pkl"
with open(output_path_config_global, 'rb') as file:
    tl_initialize_dic_global = pickle.load(file)
print(f"{output_path_config_global}数据已加载")
# SUMO配置文件路径
sumocfg_file_global = "D:\\SUMO\\work\\rebuild2\\baseline_DJL.sumocfg"
# 运行仿真
start_time_global = time.time()
MPC_counter_global=run_simulation(sumocfg_file_global,net_global,control_dic_global,G_total_dict_global,targets_dict_global,edge_search_dic_global,tl_initialize_dic_global,False,100)
end_time_global = time.time()      
elapsed_time_global = end_time_global - start_time_global
print(f"{len(seleted_indexes_list_global)}辆规模，{MPC_counter_global}车次MPC调度，共运行耗时：{elapsed_time_global}")
output_file_path_global = r"D:\\SUMO\\work\\rebuild2\\route_MPC2.pkl"  # 替换为你想要保存的文件名
with open(output_file_path_global, 'wb') as file:
    pickle.dump(control_dic_global, file)
print(f"数据已保存到：{output_file_path_global}")

# 结果数据储存

import os
import shutil

# 定义文件夹名称和文件路径
folder_name = f"DMPC_DJLBased_result{simulating_data_index}"  # 新建的文件夹名称
files_to_move =['fcd_DJL.xml','queue_DJL.xml','recorded_route_DJL.xml'] # 要移动的文件名称
source_folder = "D:\\SUMO\\work\\rebuild2\\" # 源文件夹路径
address = "D:\\SUMO\\work\\rebuild2\\"  # 新建文件夹的路径

# 创建目标文件夹（如果文件夹不存在）
target_folder = os.path.join(address, folder_name)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)
    print(f"文件夹 '{target_folder}' 创建成功。")
else:
    print(f"文件夹 '{target_folder}' 已存在。")

# 移动文件
for file_name in files_to_move:
    source_path = os.path.join(source_folder, file_name)  # 源文件路径
    destination_path = os.path.join(target_folder, file_name)  # 目标文件路径

    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"文件 '{file_name}' 已成功从 '{source_folder}' 移入文件夹 '{target_folder}'。")
    else:
        print(f"文件 '{file_name}' 在 '{source_folder}' 中不存在，无法移动。")

## DCDR在线baseline仿真与记录

In [13]:
import networkx as nx
from collections import deque

def get_edges_within_n_edges(G, start_node, n):
    """
    返回从起始节点 start_node 开始，向外扩散 n 条边所涉及的所有边。
    
    :param G: networkx.DiGraph，有向图
    :param start_node: 起始节点
    :param n: 扩散的边数
    :return: 一个列表，包含所有在 n 条边范围内的边
    """
    if start_node not in G:
        raise ValueError(f"节点 {start_node} 不在图中")

    visited = set()  # 用于存储访问过的节点
    edges = []  # 用于存储涉及的边
    queue = deque([(start_node, 0)])  # 队列，存储节点和当前扩散的边数

    while queue:
        current_node, current_distance = queue.popleft()
        
        # 如果当前节点已经在 visited 中，跳过
        if current_node in visited:
            continue
        
        # 标记当前节点为已访问
        visited.add(current_node)
        
        # 如果当前扩散的边数已经达到 n，停止扩散
        if current_distance == n:
            continue
        
        # 将当前节点的所有邻居加入队列，并增加边数
        for neighbor in G.successors(current_node):
            edges.append((current_node, neighbor))  # 记录涉及的边
            queue.append((neighbor, current_distance + 1))
    
    return edges

def coordinated_multi_vehicle_dynamic_routing(vehicles_all,paths_dict,G_dict): 
    # 算法2
    vehicle_list=[]
    action_list=[]
    for vehicle_i in vehicles_all:
        vehicle_list.append(vehicle_i)
        action_list.append(list(range(0,len(paths_dict[vehicle_i]))))
    combinations_all,_ = random_sample_action_combinations(len(vehicle_list), action_list, 100)
    time_combinations=[]
#     print(f"{len(combinations_all)} combinations are sampled with {len(vehicle_list)} vehicles and action list {action_list} ")
    for combination_i in combinations_all:
        sum_time=0
        for index_i in range(0,len(vehicle_list)):
            vehicle_id=vehicle_list[index_i]
            action_i_forward=paths_dict[vehicle_id][combination_i[index_i]]
            action_i=list(reversed(action_i_forward))
#             print(f"the {index_i}th vehicle {vehicle_id} chose the {combination_i[index_i]}th path in the sampled routes{combination_i}")
#             print(f"chosen path {action_i_forward}")
            updated_G=G_dict[vehicle_id]
            for index_i_path in range(1,len(action_i)):
                node_from=action_i[index_i_path]
                node_to=action_i[index_i_path-1]
#                 print(f"node from {node_from} to {node_to}")
                sum_time=sum_time+updated_G.edges[(node_from,node_to)]['roadtime']
        time_combinations.append(sum_time/len(vehicle_list)) # 平均时间
    min_value = min(time_combinations)
    min_index = time_combinations.index(min_value)
    combination_optimal=combinations_all[min_index]
    optimal_routes={}
    for index_i_1 in range(0,len(vehicle_list)):
        vehicle_i_1=vehicle_list[index_i_1]
        optimal_routes[vehicle_i_1]=paths_dict[vehicle_i_1][combination_i[index_i_1]]
    return optimal_routes
def initialize_traffic_lights_randomly(tl_id):
    """
    随机初始化指定交通灯的初始相位。
    
    参数:
    tl_id (str): 交通灯的ID
    """
    # 获取交通灯的所有相位信息
    logics = traci.trafficlight.getAllProgramLogics(tl_id)
    logic = logics[0]  # 获取第一个控制方案
    phases = logic.phases  # 获取所有相位
    
    # 随机选择一个相位作为初始相位
    initial_phase_index = random.randint(0, len(phases) - 1)
    traci.trafficlight.setPhase(tl_id, initial_phase_index)
    return initial_phase_index

def initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic):
    traci.trafficlight.setPhase(tl_id, tl_initialize_dic[tl_id])
    return tl_initialize_dic[tl_id]

def run_simulation(sumocfg_file,net,control_dic,G_initial,targets_dict,tl_initialize_dic,random_flag=True):
    """
    运行SUMO仿真。
    
    参数:
    sumocfg_file (str): SUMO配置文件的路径
    """
    start_time = time.time()
    # 启动SUMO仿真
#     traci.start(['sumo-gui', '-c', sumocfg_file])
    traci.start(['sumo-gui', '-c', sumocfg_file, '--step-length', '0.1'])
    # 获取所有交通灯的ID
    tls_ids = traci.trafficlight.getIDList()
    
    # 随机初始化每个交通灯的初始相位
    DealedTL_dic={}
    for tl_id in tls_ids:
        # 初始化数据字典
        key_tl,value_tl=trafficlight_dic_generation(tl_id)
#         print(f'trafficlight id: {tl_id}, applied id: {key_tl}')
        DealedTL_dic[key_tl]=value_tl
        if random_flag:
            initialize_traffic_lights_randomly(tl_id)
        else:
            initialize_traffic_lights_userdefined(tl_id,tl_initialize_dic)
    # 运行仿真
    step = 0
    MPC_counter=0
    dynamic_paths={}
    edge_id_last={}
    G_list_distributed={}
    while traci.simulation.getMinExpectedNumber() > 0:
        next_target_flag=False
        vehicles_all=traci.vehicle.getIDList()
        # 检查所有车辆是否执行至决策点，重置对应动作
        for vehicle_i_1 in vehicles_all:
            edge_id_1 = traci.vehicle.getRoadID(vehicle_i_1)
            if edge_id_1==':11_0':
                edge_id_1='11-10'
            elif edge_id_1==':10_8':
                edge_id_1='10-11'
            # 在车辆移动过程中记录实际移动的路线
            if not edge_id_1.startswith(":") and (edge_id_1 not in control_dic[vehicle_i_1]['recorded route'] or edge_id_1!=edge_id_last[vehicle_i_1]):
                control_dic[vehicle_i_1]['uk']=None
                control_dic[vehicle_i_1]['recorded route'].append(edge_id_1)
                edge_id_last[vehicle_i_1]=edge_id_1
                if edge_id_1==targets_dict[vehicle_i_1][control_dic[vehicle_i_1]['stage']]:
                    next_target_flag=True
                    control_dic[vehicle_i_1]['stage']=control_dic[vehicle_i_1]['stage']+1
                    # 若移动至最后一个终点路段，则视为到达终点，在调度过程中忽略该车辆
                    if control_dic[vehicle_i_1]['stage']>=len(targets_dict[vehicle_i_1]):
                        print(f"deleted vehicle { vehicle_i_1} with finished traveling")
                        traci.vehicle.remove(vehicle_i_1)
            else:
                continue
        if step % 100 == 0 or next_target_flag:
            vehicles_all_updated=traci.vehicle.getIDList()
            # 算法1
            dynamic_vehicles=[]
            for vehicle_i_2 in vehicles_all_updated:
                edge_id_2 = traci.vehicle.getRoadID(vehicle_i_2)
                if edge_id_2==':11_0':
                    edge_id_2='11-10'
                elif edge_id_2==':10_8':
                    edge_id_2='10-11'
                if control_dic[vehicle_i_2]['uk']==None and not edge_id_2.startswith(":"):
#                     paths_i,_,path_minT=calculate_filted_path(G_initial, edge_id_2, targets_dict[vehicle_i_2][control_dic[vehicle_i_2]['stage']],1.2,6,1.2)
                    # 更新节点属性,受限于分布式架构，对车辆i所在alpha节点而言，仅相邻Np范围内的节点信息已知
                    G_vehicle_i=G_initial.copy()
                    edges_vehicle_i=get_edges_within_n_edges(G_initial, edge_id_2, 2)
                    for connection_i in edges_vehicle_i:
                        edge_id_current=connection_i[0]
                        vehicles_current = traci.edge.getLastStepVehicleIDs(edge_id_current)
                        num_i=len(list(vehicles_current))
                        C_i=G_initial.edges[connection_i]['roadlength']/(4+6)*2 # 公式（2）取2车道平均
                        G_vehicle_i.edges[connection_i]['roadtime']=G_initial.edges[connection_i]['roadtime']*(1+0.15*(num_i/C_i)**4) #公式（1）
                    G_list_distributed[vehicle_i_2]=G_vehicle_i
                    paths_i,_,path_minT=calculate_filted_path(G_list_distributed[vehicle_i_2], edge_id_2, targets_dict[vehicle_i_2][control_dic[vehicle_i_2]['stage']],1,0,1)
                    dynamic_paths[vehicle_i_2]=[path_minT]
#                     print(f"vehicle {vehicle_i_2} generated paths {paths_i}")
                    dynamic_vehicles.append(vehicle_i_2)
                else:
                    continue
            if len(dynamic_vehicles)>0:
                optimal_routes=coordinated_multi_vehicle_dynamic_routing(dynamic_vehicles,dynamic_paths,G_list_distributed)
#                 print(f"vehicles {dynamic_vehicles} are cooperating for dynamic routes")
                MPC_counter=MPC_counter+1
            # 执行决策动作
            for vehicle_i_3 in dynamic_vehicles:
                edge_id_3 = traci.vehicle.getRoadID(vehicle_i_3)
                current_route=optimal_routes[vehicle_i_3]
                if edge_id_3==':11_0':
                    edge_id_3='11-10'
                elif edge_id_3==':10_8':
                    edge_id_3='10-11'
                traci.vehicle.setRoute(vehicle_i_3, current_route)
#             clear_output(wait=True)
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        step += 1
    end_time = time.time()      
    elapsed_time = end_time - start_time
    print(f"耗时{elapsed_time}秒")
    # 关闭TraCI连接
    traci.close()
    print(f"SUMO终端已关闭")
    return MPC_counter

# 选择初始相位数据组
simulating_data_index=14


#初始化图
net_global = sumolib.net.readNet("D:\\SUMO\\work\\rebuild2\\sourceCICT.net.xml")
G_initial=convert_net_to_digraph_simple(net_global)
seleted_indexes_list_global=database_baseline['Routes index list']
# print(seleted_indexes_list)
G_total_dict_global={}
targets_dict_global={}
vehicle_ids_global=[]
DJT_dict_global={}
for iteration_index_global in range(len(seleted_indexes_list_global)):
    vehicle_index_global=seleted_indexes_list_global[iteration_index_global]
    vehicle_id_global=str(iteration_index_global)
    current_data_index_global=seleted_indexes_list_global[vehicle_index_global]
    G_list_global=database_dic[vehicle_index_global]['Graph list']
    G_total_dict_global[vehicle_id_global]=G_list_global
    indexed_target_single_global=database_dic[vehicle_index_global]['Indexed Targets']
    targets_dict_global[vehicle_id_global]=indexed_target_single_global
    vehicle_ids_global.append(vehicle_id_global)  
    DJTpath_single_forward=database_dic[vehicle_index_global]['Possible DJT']
    DJT_dic_single={}
    DJT_dic_single['path']=DJTpath_single_forward
    DJT_dic_single['length']={}
    length_all=0
    DJTpath_single=list(reversed(DJTpath_single_forward))
    for index_i_path in range(1,len(DJTpath_single)):
        node_i_path_from=DJTpath_single[index_i_path]
        node_i_path_to=DJTpath_single[index_i_path-1]
        DJT_dic_single['length'][node_i_path_from]=length_all
        length_all=length_all+G_initial.edges[(node_i_path_from,node_i_path_to)]['roadlength']
    DJT_dict_global[vehicle_id_global]=DJT_dic_single
    


control_dic_global=control_dictionary_initialize(G_total_dict_global,targets_dict_global,vehicle_ids_global)
# 交通灯初始配置
# output_path_config_global= r"D:\\SUMO\\work\\rebuild2\\InitialPhases2.pkl" 
output_path_config_global= f"D:\\SUMO\\work\\rebuild2\\InitialPhases{simulating_data_index}.pkl"
with open(output_path_config_global, 'rb') as file:
    tl_initialize_dic_global = pickle.load(file)
print(f"{output_path_config_global}数据已加载")
# SUMO配置文件路径
sumocfg_file_global = "D:\\SUMO\\work\\rebuild2\\baseline_DJL.sumocfg"
# 运行仿真
start_time_global = time.time()
MPC_counter_global=run_simulation(sumocfg_file_global,net_global,control_dic_global,G_initial,targets_dict_global,tl_initialize_dic_global,False)
end_time_global = time.time()      
elapsed_time_global = end_time_global - start_time_global
print(f"{len(seleted_indexes_list_global)}辆规模，{MPC_counter_global}车次动态调度，共运行耗时：{elapsed_time_global}")
output_file_path_global = r"D:\\SUMO\\work\\rebuild2\\route_CCDR.pkl"  # 替换为你想要保存的文件名
with open(output_file_path_global, 'wb') as file:
    pickle.dump(control_dic_global, file)
print(f"数据已保存到：{output_file_path_global}")

# 结果数据储存

import os
import shutil

# 定义文件夹名称和文件路径
folder_name = f"CCDR_DJLBased_result{simulating_data_index}"  # 新建的文件夹名称
files_to_move =['fcd_DJL.xml','queue_DJL.xml','recorded_route_DJL.xml'] # 要移动的文件名称
source_folder = "D:\\SUMO\\work\\rebuild2\\" # 源文件夹路径
address = "D:\\SUMO\\work\\rebuild2\\"  # 新建文件夹的路径

# 创建目标文件夹（如果文件夹不存在）
target_folder = os.path.join(address, folder_name)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)
    print(f"文件夹 '{target_folder}' 创建成功。")
else:
    print(f"文件夹 '{target_folder}' 已存在。")

# 移动文件
for file_name in files_to_move:
    source_path = os.path.join(source_folder, file_name)  # 源文件路径
    destination_path = os.path.join(target_folder, file_name)  # 目标文件路径

    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"文件 '{file_name}' 已成功从 '{source_folder}' 移入文件夹 '{target_folder}'。")
    else:
        print(f"文件 '{file_name}' 在 '{source_folder}' 中不存在，无法移动。")

D:\SUMO\work\rebuild2\InitialPhases14.pkl数据已加载


<ipython-input-7-65d9707a6ae5>:18: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  total_phases=traci.trafficlight.getCompleteRedYellowGreenDefinition(TL_id)[0].getPhases()


deleted vehicle 165 with finished traveling
deleted vehicle 291 with finished traveling
deleted vehicle 68 with finished traveling
deleted vehicle 85 with finished traveling
deleted vehicle 95 with finished traveling
deleted vehicle 152 with finished traveling
deleted vehicle 45 with finished traveling
deleted vehicle 236 with finished traveling
deleted vehicle 252 with finished traveling
deleted vehicle 87 with finished traveling
deleted vehicle 151 with finished traveling
deleted vehicle 5 with finished traveling
deleted vehicle 237 with finished traveling
deleted vehicle 129 with finished traveling
deleted vehicle 31 with finished traveling
deleted vehicle 14 with finished traveling
deleted vehicle 16 with finished traveling
deleted vehicle 59 with finished traveling
deleted vehicle 146 with finished traveling
deleted vehicle 33 with finished traveling
deleted vehicle 80 with finished traveling
deleted vehicle 265 with finished traveling
deleted vehicle 188 with finished traveling
d

FatalTraCIError: Connection closed by SUMO.

In [15]:
# 关闭TraCI连接
traci.close()
print("SUMO终端已关闭")

SUMO终端已关闭


# 结果转译为.rou.xml文件以便于复现，并核验结果路由

In [36]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import shutil

simulating_data_index=14
folder_name = f"DMPC_DJLBased_result{simulating_data_index}" 
source_folder = "D:\\SUMO\\work\\rebuild2\\" # 源文件夹路径
target_folder = os.path.join(source_folder, folder_name)
source_file = os.path.join(target_folder, 'recorded_route_DJL.xml')
target_file=os.path.join(target_folder, 'route_DMPC.rou.xml')
# 指定文件路径
address_A = source_file
address_B = "D:\\SUMO\\work\\rebuild2\\route_DJL.rou.xml"
address_output = target_file

# 读取并解析 A.rou.xml 文件
tree_A = ET.parse(address_A)
root_A = tree_A.getroot()

# 读取并解析 B.rou.xml 文件
tree_B = ET.parse(address_B)
root_B = tree_B.getroot()

# 创建一个字典来存储 A 文件中每个车辆的 <routeDistribution> 信息
route_distribution_dict = {}
for vehicle in root_A.findall('vehicle'):
    vehicle_id = vehicle.get("id")
    route_distribution = vehicle.find("routeDistribution")
    if route_distribution is not None:
        route_distribution_dict[vehicle_id] = route_distribution

# 遍历 B 文件中的车辆，并替换 <route> 信息
for vehicle in root_B.findall('vehicle'):
    vehicle_id = vehicle.get("id")
    if vehicle_id in route_distribution_dict:
        # 删除原有的 <route> 元素
        for route in vehicle.findall("route"):
            vehicle.remove(route)
        # 添加新的 <routeDistribution> 元素
        vehicle.append(route_distribution_dict[vehicle_id])
        
# # 创建一个字典来存储 A 文件中每个车辆的最后一个 route 的 edges 信息
# route_dict = {}
# for vehicle in root_A.findall('vehicle'):
#     vehicle_id = vehicle.get("id")
#     route_distribution = vehicle.find("routeDistribution")
#     if route_distribution is not None:
#         # 获取最后一个 route 的 edges
#         last_route = route_distribution.findall("route")[-1]
#         route_edges = last_route.get("edges")
#         route_dict[vehicle_id] = route_edges

# # 遍历 B 文件中的车辆，并替换路线信息
# for vehicle in root_B.findall('vehicle'):
#     vehicle_id = vehicle.get("id")
#     if vehicle_id in route_dict:
#         route = vehicle.find("route")
#         if route is not None:
#             route.set("edges", route_dict[vehicle_id])

# 将修改后的 XML 写入新的文件
# 使用 minidom 来美化输出
rough_string = ET.tostring(root_B, 'utf-8')
reparsed = minidom.parseString(rough_string)
pretty_xml_as_string = reparsed.toprettyxml(indent="    ")

with open(address_output, 'w', encoding='utf-8') as f:
    f.write(pretty_xml_as_string)

print(f"处理完成，新的文件已保存为 {address_output}")

# 源文件路径
# 目标文件路径（或目录）
destination =os.path.join(source_folder, 'route_DMPC.rou.xml')  # 或 'path/to/destination_directory/'

shutil.copy(target_file, destination)
print(f"处理完成，文件已复制到 {destination}")

# 检查终点是否都在路径结果内
seleted_indexes_list_global=database_baseline['Routes index list']
targets_dict_global={}
total_pass_flag=True
for iteration_index_global in range(len(seleted_indexes_list_global)):
    vehicle_index_global=seleted_indexes_list_global[iteration_index_global]
    vehicle_id_global=str(iteration_index_global)
    indexed_target_single_global=database_dic[vehicle_index_global]['Indexed Targets']
    targets_dict_global[vehicle_id_global]=indexed_target_single_global
    check_pass_flag=True
    for i in indexed_target_single_global:
        if i not in route_dict[vehicle_id_global]:
            check_pass_flag=False
            break
    if not check_pass_flag:
        total_pass_flag=False
        print(f"核验失败，车辆 {vehicle_id_global} 在目标{indexed_target_single_global}中未经过目标{i}")
if total_pass_flag:
    print(f"核验通过，所有车辆均成功依次经过了所有目标")
else:
    print(f"核验失败，存在车辆未依次经过所有目标")

处理完成，新的文件已保存为 D:\SUMO\work\rebuild2\DMPC_DJLBased_result14\route_DMPC.rou.xml
处理完成，文件已复制到 D:\SUMO\work\rebuild2\route_DMPC.rou.xml
核验通过，所有车辆均成功依次经过了所有目标


In [35]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import shutil

simulating_data_index=18
folder_name = f"CCDR_DJLBased_result{simulating_data_index}" 
source_folder = "D:\\SUMO\\work\\rebuild2\\" # 源文件夹路径
target_folder = os.path.join(source_folder, folder_name)
source_file = os.path.join(target_folder, 'recorded_route_DJL.xml')
target_file=os.path.join(target_folder, 'route_DMPC.rou.xml')
# 指定文件路径
address_A = source_file
address_B = "D:\\SUMO\\work\\rebuild2\\route_DJL.rou.xml"
address_output = target_file

# 读取并解析 A.rou.xml 文件
tree_A = ET.parse(address_A)
root_A = tree_A.getroot()

# 读取并解析 B.rou.xml 文件
tree_B = ET.parse(address_B)
root_B = tree_B.getroot()

# # 创建一个字典来存储 A 文件中每个车辆的 <routeDistribution> 信息
# route_distribution_dict = {}
# for vehicle in root_A.findall('vehicle'):
#     vehicle_id = vehicle.get("id")
#     route_distribution = vehicle.find("routeDistribution")
#     if route_distribution is not None:
#         route_distribution_dict[vehicle_id] = route_distribution

# # 遍历 B 文件中的车辆，并替换 <route> 信息
# for vehicle in root_B.findall('vehicle'):
#     vehicle_id = vehicle.get("id")
#     if vehicle_id in route_distribution_dict:
#         # 删除原有的 <route> 元素
#         for route in vehicle.findall("route"):
#             vehicle.remove(route)
#         # 添加新的 <routeDistribution> 元素
#         vehicle.append(route_distribution_dict[vehicle_id])
        
# 创建一个字典来存储 A 文件中每个车辆的最后一个 route 的 edges 信息
route_dict = {}
for vehicle in root_A.findall('vehicle'):
    vehicle_id = vehicle.get("id")
    route_distribution = vehicle.find("routeDistribution")
    if route_distribution is not None:
        # 获取最后一个 route 的 edges
        last_route = route_distribution.findall("route")[-1]
        route_edges = last_route.get("edges")
        route_dict[vehicle_id] = route_edges

# 遍历 B 文件中的车辆，并替换路线信息
for vehicle in root_B.findall('vehicle'):
    vehicle_id = vehicle.get("id")
    if vehicle_id in route_dict:
        route = vehicle.find("route")
        if route is not None:
            route.set("edges", route_dict[vehicle_id])

# 将修改后的 XML 写入新的文件
# 使用 minidom 来美化输出
rough_string = ET.tostring(root_B, 'utf-8')
reparsed = minidom.parseString(rough_string)
pretty_xml_as_string = reparsed.toprettyxml(indent="    ")

with open(address_output, 'w', encoding='utf-8') as f:
    f.write(pretty_xml_as_string)

print(f"处理完成，新的文件已保存为 {address_output}")

# 源文件路径
# 目标文件路径（或目录）
destination =os.path.join(source_folder, 'route_DMPC.rou.xml')  # 或 'path/to/destination_directory/'

shutil.copy(target_file, destination)
print(f"处理完成，文件已复制到 {destination}")

# 检查终点是否都在路径结果内
seleted_indexes_list_global=database_baseline['Routes index list']
targets_dict_global={}
total_pass_flag=True
for iteration_index_global in range(len(seleted_indexes_list_global)):
    vehicle_index_global=seleted_indexes_list_global[iteration_index_global]
    vehicle_id_global=str(iteration_index_global)
    indexed_target_single_global=database_dic[vehicle_index_global]['Indexed Targets']
    targets_dict_global[vehicle_id_global]=indexed_target_single_global
    check_pass_flag=True
    for i in indexed_target_single_global:
        if i not in route_dict[vehicle_id_global]:
            check_pass_flag=False
            break
    if not check_pass_flag:
        total_pass_flag=False
        print(f"核验失败，车辆 {vehicle_id_global} 在目标{indexed_target_single_global}中未经过目标{i}")
if total_pass_flag:
    print(f"核验通过，所有车辆均成功依次经过了所有目标")
else:
    print(f"核验失败，存在车辆未依次经过所有目标")

处理完成，新的文件已保存为 D:\SUMO\work\rebuild2\CCDR_DJLBased_result18\route_DMPC.rou.xml
处理完成，文件已复制到 D:\SUMO\work\rebuild2\route_DMPC.rou.xml
核验通过，所有车辆均成功依次经过了所有目标


## 数据转化为excel

In [ ]:
import xml.etree.ElementTree as ET
import xlsxwriter
import os
import xml.etree.ElementTree as ET
import xlsxwriter
import os

# 读取xml文件，写入excel
def xmlToExcel(file_xml, file_excel):
    # 创建一个excel文件，并添加一个sheet，命名为orders
    workbook = xlsxwriter.Workbook(file_excel)
    sheet = workbook.add_worksheet('orders')
    # 设置粗体
    bold = workbook.add_format({'bold': True})
    # 先在第一行写标题，用粗体
    sheet.write('A1', u'time', bold)
    sheet.write('B1', u'id', bold)
    sheet.write('C1', u'x', bold)
    sheet.write('D1', u'y', bold)
    sheet.write('E1', u'angle', bold)
    sheet.write('F1', u'speed', bold)
    sheet.write('G1', u'lane', bold)
    sheet.write('H1', u'pos', bold)
    
    # 从第二行开始录入数据
    row = 1  # 从第一行的下一行开始写入数据

    # 使用 iterparse 逐行解析 XML 文件
    for event, elem in ET.iterparse(file_xml, events=('end',)):
        if elem.tag == 'timestep':
            time = elem.get('time')
            if row % 1000 ==0:
                print(f"time {time} writing")
            for vehicle in elem.findall('vehicle'):
                vehicle_id = vehicle.get('id')
                x_num = vehicle.get('x')
                y_num = vehicle.get('y')
                angle = vehicle.get('angle')
                speed = vehicle.get('speed')
                lane = vehicle.get('lane')
                pos = vehicle.get('pos')
                # 将每个timestep中车辆数据写入excel中
                sheet.write(row, 0, time)
                sheet.write(row, 1, vehicle_id)
                sheet.write(row, 2, x_num)
                sheet.write(row, 3, y_num)
                sheet.write(row, 4, angle)
                sheet.write(row, 5, speed)
                sheet.write(row, 6, lane)
                sheet.write(row, 7, pos)
                row += 1
            # 清理已完成的元素，释放内存
            elem.clear()

    # 关闭文件
    workbook.close()

# 设置文件路径
simulating_data_index = 4
folder_name = f"DMPC_DJLBased_result{simulating_data_index}"
source_folder = "D:\\SUMO\\work\\rebuild2\\"  # 源文件夹路径
target_folder = os.path.join(source_folder, folder_name)
xml_file_path = os.path.join(target_folder, 'fcd_DJL.xml')
excel_file_path = os.path.join(target_folder, 'fcd_total.xlsx')  # 输出Excel文件路径

# 调用函数
print("converting")
xmlToExcel(xml_file_path, excel_file_path) 

In [19]:
import xml.etree.ElementTree as ET
import xlsxwriter
import os
import xml.etree.ElementTree as ET
import xlsxwriter
import os

# 读取xml文件，写入excel
def xmlToExcel(file_xml, file_excel):
    # 创建一个excel文件，并添加一个sheet，命名为orders
    workbook = xlsxwriter.Workbook(file_excel)
    sheet = workbook.add_worksheet('orders')
    # 设置粗体
    bold = workbook.add_format({'bold': True})
    # 先在第一行写标题，用粗体
    sheet.write('A1', u'time', bold)
    sheet.write('B1', u'id', bold)
    sheet.write('C1', u'x', bold)
    sheet.write('D1', u'y', bold)
    sheet.write('E1', u'angle', bold)
    sheet.write('F1', u'speed', bold)
    sheet.write('G1', u'lane', bold)
    sheet.write('H1', u'pos', bold)
    
    # 从第二行开始录入数据
    row = 1  # 从第一行的下一行开始写入数据

    # 使用 iterparse 逐行解析 XML 文件
    for event, elem in ET.iterparse(file_xml, events=('end',)):
        if elem.tag == 'timestep':
            time = elem.get('time')
#             if row % 1000 ==0:
#                 print(f"time {time} writing")
            for vehicle in elem.findall('vehicle'):
                vehicle_id = vehicle.get('id')
                x_num = vehicle.get('x')
                y_num = vehicle.get('y')
                angle = vehicle.get('angle')
                speed = vehicle.get('speed')
                lane = vehicle.get('lane')
                pos = vehicle.get('pos')
                # 将每个timestep中车辆数据写入excel中
                sheet.write(row, 0, time)
                sheet.write(row, 1, vehicle_id)
                sheet.write(row, 2, x_num)
                sheet.write(row, 3, y_num)
                sheet.write(row, 4, angle)
                sheet.write(row, 5, speed)
                sheet.write(row, 6, lane)
                sheet.write(row, 7, pos)
                row += 1
            # 清理已完成的元素，释放内存
            elem.clear()

    # 关闭文件
    workbook.close()

indexes_select=[2,4,7,14,18]
for simulating_data_index in indexes_select:
    # 设置文件路径
    # simulating_data_index = 9
    folder_name = f"CCDR_DJLBased_result{simulating_data_index}"
    source_folder = "D:\\SUMO\\work\\rebuild2\\"  # 源文件夹路径
    target_folder = os.path.join(source_folder, folder_name)
    xml_file_path = os.path.join(target_folder, 'fcd_DJL.xml')
    excel_file_path = os.path.join(target_folder, 'fcd_total.xlsx')  # 输出Excel文件路径

    # 调用函数
    print(f"{xml_file_path} converting")
    xmlToExcel(xml_file_path, excel_file_path) 
    print(f"{xml_file_path} converted")

D:\SUMO\work\rebuild2\CCDR_DJLBased_result2\fcd_DJL.xml converting
D:\SUMO\work\rebuild2\CCDR_DJLBased_result2\fcd_DJL.xml converted
D:\SUMO\work\rebuild2\CCDR_DJLBased_result4\fcd_DJL.xml converting
D:\SUMO\work\rebuild2\CCDR_DJLBased_result4\fcd_DJL.xml converted
D:\SUMO\work\rebuild2\CCDR_DJLBased_result7\fcd_DJL.xml converting
D:\SUMO\work\rebuild2\CCDR_DJLBased_result7\fcd_DJL.xml converted
D:\SUMO\work\rebuild2\CCDR_DJLBased_result14\fcd_DJL.xml converting
D:\SUMO\work\rebuild2\CCDR_DJLBased_result14\fcd_DJL.xml converted
D:\SUMO\work\rebuild2\CCDR_DJLBased_result18\fcd_DJL.xml converting
D:\SUMO\work\rebuild2\CCDR_DJLBased_result18\fcd_DJL.xml converted


## 数据转化为.pkl

In [4]:
import xml.etree.ElementTree as ET
import xlsxwriter
import os
import xml.etree.ElementTree as ET
import xlsxwriter
import os
from scipy.io import savemat

# 读取xml文件，写入excel
def datadeal(file_xml):
    vehicle_indicator={}
    vehicle_dataframe={}
    frame_indicator={}
    vehicle_set=set()
    for event, elem in ET.iterparse(file_xml, events=('end',)):
        if elem.tag == 'timestep':
            time = elem.get('time')
            if float(time)%500==0:
                print(f"processing time {time}")
            vehicle_num_current=0
            vehicle_speed_current=0
            for vehicle in elem.findall('vehicle'):
                vehicle_id = vehicle.get('id')
                if vehicle_id not in vehicle_set:
                    vehicle_set.add(vehicle_id)
                    vehicle_dataframe[f'vehicle_{vehicle_id}']={}
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['time']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['x']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['y']=[]
                    vehicle_dataframe[f'vehicle_{vehicle_id}']['speed']=[]
                x_num = vehicle.get('x')
                y_num = vehicle.get('y')
                speed = vehicle.get('speed')
                vehicle_dataframe[f'vehicle_{vehicle_id}']['time'].append(float(time))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['x'].append(float(x_num))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['y'].append(float(y_num))
                vehicle_dataframe[f'vehicle_{vehicle_id}']['speed'].append(float(speed))
                vehicle_num_current=vehicle_num_current+1
                vehicle_speed_current=vehicle_speed_current+float(speed)
            frame_indicator[f'time_{int(float(time)*1000)}_ms']=np.array([vehicle_num_current,vehicle_speed_current/max(vehicle_num_current,1)])
            elem.clear()
    for vehicle_id_ind in vehicle_dataframe:
        start_time=min(vehicle_dataframe[vehicle_id_ind]['time'])
        end_time=max(vehicle_dataframe[vehicle_id_ind]['time'])
        total_speed=sum(vehicle_dataframe[vehicle_id_ind]['speed'])
        x_list=vehicle_dataframe[vehicle_id_ind]['x']
        diff_x=[x_list[i+1] - x_list[i] for i in range(len(x_list) - 1)]
        y_list=vehicle_dataframe[vehicle_id_ind]['y']
        diff_y=[y_list[i+1] - y_list[i] for i in range(len(y_list) - 1)]
        distance_step=[a**2 + b**2 for a, b in zip(diff_x, diff_y)]
        vehicle_indicator[vehicle_id_ind]={}
        vehicle_indicator[vehicle_id_ind]['appear']=start_time
        vehicle_indicator[vehicle_id_ind]['disappear']=end_time
        vehicle_indicator[vehicle_id_ind]['distance']=sum(distance_step)
        vehicle_indicator[vehicle_id_ind]['averaspeed']=sum(distance_step)/(end_time-start_time)
        vehicle_dataframe[vehicle_id_ind]['time']=np.array(vehicle_dataframe[vehicle_id_ind]['time'])
        vehicle_dataframe[vehicle_id_ind]['x']=np.array(vehicle_dataframe[vehicle_id_ind]['x'])
        vehicle_dataframe[vehicle_id_ind]['y']=np.array(vehicle_dataframe[vehicle_id_ind]['y'])
        vehicle_dataframe[vehicle_id_ind]['speed']=np.array(vehicle_dataframe[vehicle_id_ind]['speed'])

    # 关闭文件
    return vehicle_indicator,frame_indicator,vehicle_dataframe
# indexes_select=[11111,21111,31111,12111,13111,14111,11211,11311,11411,11121,11131,11112,11113]
indexes_select=[12111,13111,14111]
full_dealed_data={}
for simulating_data_index in indexes_select:
    # 设置文件路径
    # simulating_data_index = 9
    folder_name = f"DMPC_DJLBased_result{simulating_data_index}"
    # folder_name = f"CCDR_DJLBased_result{simulating_data_index}"
    source_folder = "D:\\SUMO\\work\\"  # 源文件夹路径
    target_folder = os.path.join(source_folder, folder_name)
    xml_file_path = os.path.join(target_folder, 'fcd_DJL.xml')
    # 调用函数
    print(f"{xml_file_path} converting")
    vehicle_indicator,frame_indicator,vehicle_dataframe=datadeal(xml_file_path)
    full_dealed_data[f'file_{simulating_data_index}']={}
    full_dealed_data[f'file_{simulating_data_index}']['vehicle_indicator']=vehicle_indicator
    full_dealed_data[f'file_{simulating_data_index}']['frame_indicator']=frame_indicator
    full_dealed_data[f'file_{simulating_data_index}']['dataframes']=vehicle_dataframe
    clear_output(wait=True)
    single_dealed_data={}
    single_dealed_data['vehicle_indicator']=vehicle_indicator
    single_dealed_data['frame_indicator']=frame_indicator
    single_dealed_data['dataframes']=vehicle_dataframe
    with open(f"D:\\SUMO\\work\\DMPC_data_{simulating_data_index}.pkl", 'wb') as file:
    # with open(f"D:\\SUMO\\work\\CCDR_data_{simulating_data_index}.pkl", 'wb') as file:
        pickle.dump(single_dealed_data, file)
    print(f"{xml_file_path} converted")


D:\SUMO\work\DMPC_DJLBased_result14111\fcd_DJL.xml converted
